In [ ]:
# Simple 1D nightingale layout + 3D UniProt mol* wrapper (left and right panels)

In [5]:
# Cell 1: Fetch data server-side and normalize it (no browser CORS)
import json, re, requests
from typing import Dict, Any, List, Tuple

SESS = requests.Session()
SESS.headers.update({"User-Agent": "nightingale-builder/1.2"})

def _get(url: str, timeout=30):
    r = SESS.get(url, timeout=timeout)
    r.raise_for_status()
    return r

def fetch_json(url: str, timeout: int = 30) -> Dict[str, Any]:
    return _get(url, timeout).json()

def fetch_features(acc: str) -> Dict[str, Any]:
    return fetch_json(f"https://www.ebi.ac.uk/proteins/api/features/{acc}")

def fetch_variation(acc: str) -> Dict[str, Any]:
    try:
        return fetch_json(f"https://www.ebi.ac.uk/proteins/api/variation/{acc}")
    except Exception:
        return {"features": []}

def fetch_uniprot(acc: str) -> Dict[str, Any]:
    return fetch_json(f"https://rest.uniprot.org/uniprotkb/{acc}.json")

def fetch_scop3p(acc: str) -> Dict[str, Any]:
    return fetch_json(f"https://iomics.ugent.be/scop3p/api/modifications?accession={acc}")

def _iter_pdb_xrefs(u: Dict[str, Any]):
    xs = u.get("uniProtKBCrossReferences") or []
    if xs:
        for x in xs:
            if (x.get("database") or x.get("type")) == "PDB":
                yield {"id": x.get("id"), "properties": x.get("properties") or []}
    legacy = u.get("dbReferences") or []
    for x in legacy:
        if x.get("type") == "PDB":
            yield {"id": x.get("id"), "properties": x.get("properties") or []}

def parse_pdb_structures(uniprot_json: Dict[str, Any]) -> List[Dict[str, Any]]:
    out: List[Dict[str, Any]] = []
    import re
    for xr in _iter_pdb_xrefs(uniprot_json):
        pdb_id = xr.get("id")
        if not pdb_id:
            continue
        props = { (p.get("type") or p.get("key")): p.get("value") for p in xr.get("properties", []) }
        chains_str = (props.get("chains") or props.get("Chains") or "").strip()
        if not chains_str:
            continue
        parts = re.split(r"[;,]\s*", chains_str)
        for part in filter(None, parts):
            m = re.match(r"^([A-Za-z0-9]+)\s*=\s*([0-9,\-\s]+)$", part)
            if not m:
                continue
            chain = m.group(1).strip()
            ranges_blob = m.group(2)
            segs: List[Tuple[int, int]] = []
            for seg in re.split(r"\s*,\s*", ranges_blob):
                if not seg: continue
                ab = seg.split("-")
                if len(ab) == 2 and ab[0].isdigit() and ab[1].isdigit():
                    segs.append((int(ab[0]), int(ab[1])))
            if not segs: continue
            out.append({
                "id": pdb_id,
                "source": "PDB",
                "chain": chain,
                "label": f"{pdb_id} — chain {chain}",
                "mappings": [{"uniprot_start": s, "uniprot_end": e} for (s, e) in segs]
            })
    return out

def merge_positions_to_ranges(positions: List[int]) -> List[Tuple[int, int]]:
    if not positions:
        return []
    s = sorted(set(int(p) for p in positions))
    ranges: List[Tuple[int,int]] = []
    start = prev = s[0]
    for p in s[1:]:
        if p == prev + 1:
            prev = p
        else:
            ranges.append((start, prev))
            start = prev = p
    ranges.append((start, prev))
    return ranges

def build_bundle(acc: str) -> Dict[str, Any]:
    features = fetch_features(acc)
    variation = fetch_variation(acc)
    uniprot = fetch_uniprot(acc)
    scop3p = fetch_scop3p(acc)

    structures = [{"id": f"AF-{acc}-F1", "source": "AFDB", "chain": None, "label": f"AlphaFold (AF-{acc}-F1)"}]
    structures.extend(parse_pdb_structures(uniprot))

    phospho = [m for m in (scop3p.get("modifications") or []) if str(m.get("name","")).lower() == "phosphorylation"]
    phospho_sites = [
        {"pos": int(m["position"]), "residue": (m.get("residue") or "?")[:1], "source": m.get("source") or "Scop3P"}
        for m in phospho if "position" in m
    ]
    highlight_ranges = merge_positions_to_ranges([p["pos"] for p in phospho_sites])
    highlight_str = ",".join(f"{s}:{e}" for (s, e) in highlight_ranges)  # colon-separated for structure

    return {
        "accession": acc,
        "features": features,
        "variation": variation,
        "structures": structures,
        "scop3p": scop3p,
        "scop3p_compact": {"phospho_sites": phospho_sites, "highlight": highlight_str},
    }


In [38]:
# Cell 2 (modified): Build a self-contained Nightingale HTML with enforced grey theme (attrs + props)
from pathlib import Path
import json

HTML_TEMPLATE = """<!doctype html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width, initial-scale=1"/>
<title>Nightingale — __ACC__</title>
<style>
  :root { --panel-height: 560px; }
  body { margin: 0; font-family: system-ui, -apple-system, Segoe UI, Roboto, sans-serif; }
  .grid { display: grid; grid-template-columns: 1fr 1fr; gap: 16px; padding: 16px; align-items: stretch; }
  .left, .right { height: var(--panel-height); min-height: var(--panel-height); }
  .left { overflow: auto; border-right: 1px solid #e5e7eb; padding-right: 8px; }
  .right { display: flex; flex-direction: column; }
  .panel { flex: 1 1 auto; display: flex; flex-direction: column; gap: 8px; }
  .controls { display: flex; gap: 12px; align-items: center; flex-wrap: wrap; }
  label { font-size: 0.9rem; color: #374151; }
  .hint { color: #374151; font-size: 0.9rem; padding: 0 16px; }
  .warn { color: #b45309; font-size: 0.85rem; padding: 4px 16px 0; }
  table { border-collapse: collapse; width: 100%; }
  td { padding: 5px; }
  td:first-child { background-color: lightcyan; font: 0.8em sans-serif; white-space: nowrap; }
  td:nth-child(2) { background-color: aliceblue; }
  tr:nth-child(-n + 3) > td { background-color: transparent; }
  nightingale-structure { display: block; width: 100%; height: 100%; border: 1px solid #e5e7eb; border-radius: 8px; }
  .legend { font-size: 12px; color: #374151; margin: 6px 0; }
</style>

<!-- Mol* polyfills required by nightingale-structure (before any module loads) -->
<script>
  window.global = window;
  window.process = window.process || { env: {} };
</script>

<script type="importmap">
{
  "imports": {
    "@nightingale-elements/": "https://cdn.jsdelivr.net/npm/@nightingale-elements/"
  }
}
</script>

<!-- Embedded caches (built in Python; avoids browser CORS) -->
<script id="features-cache"   type="application/json">__FEATURES_CACHE__</script>
<script id="variation-cache"  type="application/json">__VARIATION_CACHE__</script>
<script id="structures-cache" type="application/json">__STRUCTURES_CACHE__</script>
<script id="scop3p-cache"     type="application/json">__SCOP3P_CACHE__</script>
<script id="scop3p-compact"   type="application/json">__SCOP3P_COMPACT__</script>
</head>

<body>
<div class="hint">1D left · 3D right. Click in 1D to select; Scop3P overlay is toggleable. PDB highlights auto-clamped to mapped ranges.</div>
<div class="warn" id="warn"></div>
<div class="grid">
  <!-- 1D (left) -->
  <div class="left">
    <nightingale-manager id="mgr">
      <table>
        <tbody>
          <tr>
            <td></td>
            <td>
              <nightingale-navigation id="navigation"
                min-width="800" height="40" length="770" display-start="1" display-end="770"
                margin-color="white" highlight-event="onclick"
              ></nightingale-navigation>
            </td>
          </tr>
          <tr>
            <td></td>
            <td>
              <nightingale-sequence id="sequence"
                min-width="800" height="40" length="770" display-start="1" display-end="770"
                margin-color="white" highlight-event="onclick"
              ></nightingale-sequence>
            </td>
          </tr>
          <tr>
            <td></td>
            <td>
              <nightingale-colored-sequence id="colored-sequence"
                min-width="800" height="15" length="770" display-start="1" display-end="770"
                scale="hydrophobicity-scale" margin-color="white" highlight-event="onclick"
              ></nightingale-colored-sequence>
            </td>
          </tr>
          <tr>
            <td>Domain</td>
            <td>
              <nightingale-track id="domain"
                min-width="800" height="18" length="770" display-start="1" display-end="770"
                margin-color="aliceblue" highlight-event="onclick" display-labels="true"
              ></nightingale-track>
              <div id="domain-legend" class="legend"></div>
            </td>
          </tr>
          <tr><td>Region</td><td><nightingale-track id="region" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Site</td><td><nightingale-track id="site"   min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Chain</td><td><nightingale-track id="chain"  layout="non-overlapping" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Binding site</td><td><nightingale-track id="binding" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Disulfide bond</td><td><nightingale-track id="disulfide-bond" layout="non-overlapping" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Beta strand</td><td><nightingale-track id="beta-strand" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Scop3P phosphorylation</td><td><nightingale-track id="scop3p" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Variants</td><td><nightingale-linegraph-track id="variants" min-width="800" height="60" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-linegraph-track></td></tr>
        </tbody>
      </table>
    </nightingale-manager>
  </div>

  <!-- 3D (right) -->
  <div class="right">
    <div class="panel">
      <div class="controls">
        <label>Structure
          <select id="structureSelect" style="margin-left:6px;padding:4px 6px;"></select>
        </label>
        <label style="margin-left:12px;">
          <input type="checkbox" id="toggleScop3P" checked>
          Show Scop3P sites
        </label>
      </div>
      <nightingale-structure
        id="structure"
        protein-accession="__ACC__"
        structure-id="AF-__ACC__-F1"
        color-theme="uniform"
        color-value="#9ca3af"
        highlight="__HIGHLIGHT__"
        highlight-color="red"
        style="--custom-structure-height: var(--panel-height)"
      ></nightingale-structure>
    </div>
  </div>
</div>

<script type="module">
  import "@nightingale-elements/nightingale-sequence@latest";
  import "@nightingale-elements/nightingale-track@latest";
  import "@nightingale-elements/nightingale-manager@latest";
  import "@nightingale-elements/nightingale-navigation@latest";
  import "@nightingale-elements/nightingale-colored-sequence@latest";
  import "@nightingale-elements/nightingale-linegraph-track@latest";
  import "@nightingale-elements/nightingale-structure@latest";

  const accession = "__ACC__";
  const featuresCache   = JSON.parse(document.getElementById("features-cache").textContent || "{}");
  const variationCache  = JSON.parse(document.getElementById("variation-cache").textContent || "{}");
  const structuresCache = JSON.parse(document.getElementById("structures-cache").textContent || "{}");
  const scop3pCache     = JSON.parse(document.getElementById("scop3p-cache").textContent || "{}");
  const compScopAll     = JSON.parse(document.getElementById("scop3p-compact").textContent || "{}");

  const warnEl = document.getElementById("warn");
  function setWarn(msg){ warnEl.textContent = msg || ""; }

  const compScop        = compScopAll[accession] || {phospho_sites:[], highlight:""};
  const featuresData    = featuresCache[accession];
  const variationData   = variationCache[accession] || { features: [] };
  const structures      = structuresCache[accession] || [{ id: `AF-${accession}-F1`, source:"AFDB", chain:null, label:`AlphaFold (AF-${accession}-F1)`, mappings:[{uniprot_start:1,uniprot_end:featuresData.sequence.length}] }];

  await Promise.all([
    customElements.whenDefined("nightingale-sequence"),
    customElements.whenDefined("nightingale-navigation"),
    customElements.whenDefined("nightingale-manager"),
    customElements.whenDefined("nightingale-track"),
    customElements.whenDefined("nightingale-colored-sequence"),
    customElements.whenDefined("nightingale-linegraph-track"),
    customElements.whenDefined("nightingale-structure"),
  ]);

  // DOM
  const mgr = document.getElementById("mgr");
  const navEl = document.getElementById("navigation");
  const seqEl = document.getElementById("sequence");
  const coloredEl = document.getElementById("colored-sequence");
  const trDomain = document.getElementById("domain");
  const domainLegend = document.getElementById("domain-legend");
  const trRegion = document.getElementById("region");
  const trSite = document.getElementById("site");
  const trBinding = document.getElementById("binding");
  const trChain = document.getElementById("chain");
  const trDisulf = document.getElementById("disulfide-bond");
  const trStrand = document.getElementById("beta-strand");
  const trScop3P = document.getElementById("scop3p");
  const variantsEl = document.getElementById("variants");
  const structureEl = document.getElementById("structure");
  const structSelect = document.getElementById("structureSelect");
  const toggleScop = document.getElementById("toggleScop3P");

  // Sizes
  const seq = featuresData.sequence;
  const len = seq.length;
  const setLen = (el) => { if (!el) return; el.length = len; el.setAttribute("length", String(len)); el.displayStart = 1; el.displayEnd = len; el.setAttribute("display-start","1"); el.setAttribute("display-end", String(len)); };
  [navEl, seqEl, coloredEl, trDomain, trRegion, trSite, trBinding, trChain, trDisulf, trStrand, trScop3P, variantsEl].forEach(setLen);

  // Sequence
  seqEl.data = seq;
  coloredEl.data = seq;

  // Features
  const feats = (featuresData.features || []).map((ft) => ({ ...ft, start: ft.start ?? ft.begin }));
  trDomain.data = feats.filter(({ type }) => type === "DOMAIN").map(f => ({ ...f, label: f.description || f.type || "domain" }));
  trRegion.data = feats.filter(({ type }) => type === "REGION");
  trSite.data   = feats.filter(({ type }) => type === "SITE");
  trBinding.data= feats.filter(({ type }) => type === "BINDING");
  trChain.data  = feats.filter(({ type }) => type === "CHAIN");
  trDisulf.data = feats.filter(({ type }) => type === "DISULFID");
  trStrand.data = feats.filter(({ type }) => type === "STRAND");

  // Domain legend fallback
  try {
    const items = trDomain.data.slice(0, 12).map(d => `${d.label || d.description || "domain"} (${d.start}-${d.end})`);
    domainLegend.textContent = items.length ? "Domains: " + items.join(" · ") + (trDomain.data.length > 12 ? " …" : "") : "";
  } catch(_) {}

  // Variants
  const counts = new Map();
  for (const v of (variationData.features || [])) {
    const pos = Number(v.begin);
    if (!Number.isFinite(pos)) continue;
    counts.set(pos, (counts.get(pos) || 0) + 1);
  }
  const values = Array.from(counts, ([position, value]) => ({ position, value })).sort((a,b)=>a.position-b.position);
  const max = values.length ? Math.max(...values.map(d=>d.value)) : 0;
  variantsEl.data = [{ color: "grey", values, range: [0, Math.max(1, max)] }];

  // Scop3P track (single-residue features)
  trScop3P.data = (scop3pCache[accession]?.modifications || [])
    .filter(m => String(m.name).toLowerCase()==="phosphorylation")
    .filter(m => Number.isFinite(Number(m.position)))
    .map(m => ({ start: Number(m.position), end: Number(m.position), type: "SCOP3P_PHOSPHO", description: `Phospho ${(m.residue||'?').slice(0,1)}@${m.position} (${m.source||'Scop3P'})` }));

  // Build structure selector (AF first, then PDB+chain)
  function populateStructureSelect(arr) {
    structSelect.innerHTML = "";
    for (const s of arr) {
      const opt = document.createElement("option");
      opt.value = (s.source === "PDB" && s.chain) ? `${s.id}|${s.chain}` : s.id;
      opt.textContent = s.label || (s.source === "PDB" ? `${s.id} — chain ${s.chain||'?'}` : s.id);
      // store mappings on option for quick access
      opt.dataset.mappings = JSON.stringify(s.mappings || []);
      opt.dataset.source = s.source || "";
      opt.dataset.chain = s.chain || "";
      structSelect.appendChild(opt);
    }
    const afIdx = Array.from(structSelect.options).findIndex(o => o.value.startsWith(`AF-${accession}-F1`));
    structSelect.selectedIndex = afIdx >= 0 ? afIdx : 0;
  }
  populateStructureSelect(structures);

  // ----- range helpers -----
  function parseRanges(str) {
    if (!str) return [];
    return str.split(",").map(x => x.trim()).filter(Boolean).map(part => {
      const [a,b] = part.split(":").map(n => Number(n));
      if (!Number.isFinite(a) || !Number.isFinite(b)) return null;
      const s = Math.min(a,b), e = Math.max(a,b);
      return [s,e];
    }).filter(Boolean).sort((r1,r2)=>r1[0]-r2[0] || r1[1]-r2[1]);
  }
  function rangesToString(ranges) {
    return ranges.map(([s,e]) => `${s}:${e}`).join(",");
  }
  function mergeRanges(ranges) {
    if (!ranges.length) return [];
    ranges.sort((a,b)=>a[0]-b[0] || a[1]-b[1]);
    const out = [];
    let [cs, ce] = ranges[0];
    for (let i=1;i<ranges.length;i++){
      const [s,e] = ranges[i];
      if (s <= ce + 1) ce = Math.max(ce, e);
      else { out.push([cs, ce]); cs = s; ce = e; }
    }
    out.push([cs, ce]);
    return out;
  }
  function unionRanges(aStr, bStr) {
    const arr = [...parseRanges(aStr), ...parseRanges(bStr)];
    return rangesToString(mergeRanges(arr));
  }
  function intersectWithAllowed(highlightStr, allowedRanges) {
    const hs = parseRanges(highlightStr);
    if (!allowedRanges || !allowedRanges.length) return rangesToString(mergeRanges(hs)); // no constraints
    const out = [];
    for (const [hs1,he1] of hs){
      for (const {uniprot_start:as1, uniprot_end:ae1} of allowedRanges){
        const s = Math.max(hs1, Number(as1));
        const e = Math.min(he1, Number(ae1));
        if (s <= e) out.push([s,e]);
      }
    }
    return rangesToString(mergeRanges(out));
  }

  // ----- sticky highlight: Scop3P (toggleable fixed) + user selection -----
  const fixedScop = (compScop.highlight || ""); // colon-separated
  let showScop = true;                           // checkbox state
  let userHighlight = "";                        // from navigation
  function currentFixed() { return showScop ? fixedScop : ""; }

  // Enforce uniform grey base (set attrs + props)
  function enforceGrey() {
    structureEl.setAttribute("color-theme", "uniform");
    structureEl.setAttribute("color-value", "#9ca3af");
    // Also set JS props (some builds listen to props, not attrs)
    structureEl.colorTheme = "uniform";
    structureEl.colorValue = "#9ca3af";
    structureEl.requestUpdate?.();
  }

  // Core apply (clamps to mapping for PDB; applies grey + red overlay)
  function applyStructure(option) {
    const value = option.value;
    const source = option.dataset.source || "";
    const mappings = JSON.parse(option.dataset.mappings || "[]");

    const merged = unionRanges(currentFixed(), userHighlight);
    const clamped = (source === "PDB" && mappings.length)
      ? intersectWithAllowed(merged, mappings)
      : merged; // AFDB/no mapping => no clamp

    if (merged && !clamped) {
      setWarn("Selected PDB-chain doesn’t cover the chosen residues; highlight was omitted.");
    } else {
      setWarn("");
    }

    let structureId = value.includes("|") ? value.split("|")[0] : value;
    structureEl.setAttribute("protein-accession", accession);
    structureEl.setAttribute("structure-id", structureId);

    enforceGrey(); // ensure grey base

    structureEl.setAttribute("highlight", clamped);
    structureEl.setAttribute("highlight-color", "red");
  }

  // Reassert visuals briefly after changes (covers late resets)
  function reassert(option, times = 6, delay = 200) {
    let n = 0;
    const id = setInterval(() => {
      n++;
      enforceGrey();
      const source = option.dataset.source || "";
      const mappings = JSON.parse(option.dataset.mappings || "[]");
      const merged = unionRanges(currentFixed(), userHighlight);
      const clamped = (source === "PDB" && mappings.length)
        ? intersectWithAllowed(merged, mappings)
        : merged;
      structureEl.setAttribute("highlight", clamped);
      structureEl.setAttribute("highlight-color", "red");
      if (n >= times) clearInterval(id);
    }, delay);
  }

  // Initial apply
  const currentOpt = structSelect.options[structSelect.selectedIndex];
  applyStructure(currentOpt);
  reassert(currentOpt);

  // Dropdown change
  structSelect.addEventListener("change", () => {
    const opt = structSelect.options[structSelect.selectedIndex];
    applyStructure(opt);
    reassert(opt);
  });

  // Toggle Scop3P
  toggleScop.addEventListener("change", () => {
    showScop = !!toggleScop.checked;
    const opt = structSelect.options[structSelect.selectedIndex];
    applyStructure(opt);
    reassert(opt, 3, 180);
  });

  // Update from 1D clicks (navigation reflects selection)
  const obs = new MutationObserver(() => {
    userHighlight = navEl.getAttribute("highlight") || "";
    const opt = structSelect.options[structSelect.selectedIndex];
    applyStructure(opt);
    reassert(opt, 2, 150);
  });
  obs.observe(navEl, { attributes: true, attributeFilter: ["highlight"] });

  mgr.addEventListener("click", () => {
    userHighlight = navEl.getAttribute("highlight") || "";
    const opt = structSelect.options[structSelect.selectedIndex];
    applyStructure(opt);
  });
</script>
</body>
</html>
"""

def build_html(acc: str, out_dir: str = ".") -> Path:
    b = build_bundle(acc)
    features_cache   = json.dumps({acc: b["features"]}, ensure_ascii=False)
    variation_cache  = json.dumps({acc: b["variation"]}, ensure_ascii=False)
    structures_cache = json.dumps({acc: b["structures"]}, ensure_ascii=False)
    scop3p_cache     = json.dumps({acc: b["scop3p"]}, ensure_ascii=False)
    scop3p_compact   = json.dumps({acc: b["scop3p_compact"]}, ensure_ascii=False)

    html_out = (
        HTML_TEMPLATE
        .replace("__ACC__", acc)
        .replace("__FEATURES_CACHE__", features_cache)
        .replace("__VARIATION_CACHE__", variation_cache)
        .replace("__STRUCTURES_CACHE__", structures_cache)
        .replace("__SCOP3P_CACHE__", scop3p_cache)
        .replace("__SCOP3P_COMPACT__", scop3p_compact)
        .replace("__HIGHLIGHT__", b["scop3p_compact"]["highlight"])
    )
    path = Path(out_dir) / f"nightingale_{acc}.html"
    path.write_text(html_out, encoding="utf-8")
    return path

# Example:
# p = build_html("P05067"); p


In [7]:
p = build_html("P07949")
p



PosixPath('nightingale_P07949.html')

In [ ]:
#python -m http.server 8000
# open http://localhost:8000/nightingale_P05067.html


In [8]:
#### Directly use PDBe-Mol* for 3D instead of Nightingale wrapper

In [7]:
# Cell 1: Fetch data server-side and normalize it (no browser CORS)
import json, re, requests
from typing import Dict, Any, List, Tuple

SESS = requests.Session()
SESS.headers.update({"User-Agent": "nightingale-pdbe-builder/1.1"})

def _get(url: str, timeout=30):
    r = SESS.get(url, timeout=timeout)
    r.raise_for_status()
    return r

def fetch_json(url: str, timeout: int = 30) -> Dict[str, Any]:
    return _get(url, timeout).json()

def fetch_features(acc: str) -> Dict[str, Any]:
    return fetch_json(f"https://www.ebi.ac.uk/proteins/api/features/{acc}")

def fetch_variation(acc: str) -> Dict[str, Any]:
    try:
        return fetch_json(f"https://www.ebi.ac.uk/proteins/api/variation/{acc}")
    except Exception:
        return {"features": []}

def fetch_uniprot(acc: str) -> Dict[str, Any]:
    return fetch_json(f"https://rest.uniprot.org/uniprotkb/{acc}.json")

def fetch_scop3p(acc: str) -> Dict[str, Any]:
    return fetch_json(f"https://iomics.ugent.be/scop3p/api/modifications?accession={acc}")

def _iter_pdb_xrefs(u: Dict[str, Any]):
    xs = u.get("uniProtKBCrossReferences") or []
    if xs:
        for x in xs:
            if (x.get("database") or x.get("type")) == "PDB":
                yield {"id": x.get("id"), "properties": x.get("properties") or []}
    legacy = u.get("dbReferences") or []
    for x in legacy:
        if x.get("type") == "PDB":
            yield {"id": x.get("id"), "properties": x.get("properties") or []}

def parse_pdb_structures(uniprot_json: Dict[str, Any]) -> List[Dict[str, Any]]:
    """
    Return unique PDB items: either per-chain (when UniProt gives chain mapping text) or
    a plain PDB entry when no chain text is available.
    """
    seen = set()
    out: List[Dict[str, Any]] = []
    for xr in _iter_pdb_xrefs(uniprot_json):
        pdb_id = xr.get("id")
        if not pdb_id:
            continue
        props = { (p.get("type") or p.get("key")): p.get("value") for p in xr.get("properties", []) }
        chains_str = (props.get("chains") or props.get("Chains") or "").strip()
        added_chain = False
        if chains_str:
            # Parse "A=25-300, 310-350; B=..." style
            parts = re.split(r"[;,]\s*", chains_str)
            for part in filter(None, parts):
                m = re.match(r"^([A-Za-z0-9]+)\s*=\s*([0-9,\-\s]+)$", part)
                if not m:
                    continue
                chain = m.group(1).strip()
                key = (pdb_id.upper(), chain)
                if key in seen:
                    continue
                seen.add(key)
                out.append({
                    "id": pdb_id.upper(),
                    "source": "PDB",
                    "chain": chain,
                    "label": f"{pdb_id.upper()} — chain {chain}",
                })
                added_chain = True
        if not added_chain:
            # Keep a plain entry (no chain mapping disclosed by UniProt)
            key = (pdb_id.upper(), None)
            if key not in seen:
                seen.add(key)
                out.append({
                    "id": pdb_id.upper(),
                    "source": "PDB",
                    "chain": None,
                    "label": pdb_id.upper(),
                })
    # Sort AF/PDB later; here just by id/chain
    out.sort(key=lambda d: (d["id"], "" if d["chain"] is None else str(d["chain"])))
    return out

def merge_positions_to_ranges(positions: List[int]) -> List[Tuple[int, int]]:
    if not positions:
        return []
    s = sorted(set(int(p) for p in positions))
    ranges: List[Tuple[int,int]] = []
    start = prev = s[0]
    for p in s[1:]:
        if p == prev + 1:
            prev = p
        else:
            ranges.append((start, prev))
            start = prev = p
    ranges.append((start, prev))
    return ranges

def build_bundle(acc: str) -> Dict[str, Any]:
    features = fetch_features(acc)
    variation = fetch_variation(acc)
    uniprot = fetch_uniprot(acc)
    scop3p = fetch_scop3p(acc)

    # AlphaFold first
    structures = [{"id": f"AF-{acc}-F1", "source": "AFDB", "chain": None, "label": f"AlphaFold (AF-{acc}-F1)"}]
    structures.extend(parse_pdb_structures(uniprot))

    phospho = [m for m in (scop3p.get("modifications") or []) if str(m.get("name","")).lower() == "phosphorylation"]
    phospho_sites = [
        {"pos": int(m["position"]), "residue": (m.get("residue") or "?")[:1], "source": m.get("source") or "Scop3P"}
        for m in phospho if "position" in m
    ]
    highlight_ranges = merge_positions_to_ranges([p["pos"] for p in phospho_sites])
    highlight_str = ",".join(f"{s}:{e}" for (s, e) in highlight_ranges)

    return {
        "accession": acc,
        "features": features,
        "variation": variation,
        "structures": structures,
        "scop3p": scop3p,
        "scop3p_compact": {"phospho_sites": phospho_sites, "highlight": highlight_str},
    }


In [4]:
# # Cell 2: Build HTML (1D Nightingale left, PDBe Mol* right) — PDB fetch fix + bounded viewer
# from pathlib import Path
# import json

# HTML_TEMPLATE = """<!doctype html>
# <html lang="en">
# <head>
# <meta charset="utf-8"/>
# <meta name="viewport" content="width=device-width, initial-scale=1"/>
# <title>Nightingale + PDBe Mol* — __ACC__</title>

# <style>
#   :root { --panel-height: 560px; }
#   html, body { height: auto !important; overflow: auto; }
#   body { margin: 0; font-family: system-ui, -apple-system, Segoe UI, Roboto, sans-serif; }
#   .grid { display: grid; grid-template-columns: 1fr 1fr; gap: 16px; padding: 16px; align-items: stretch; }
#   .left, .right { height: var(--panel-height); min-height: var(--panel-height); }
#   .left { overflow: auto; border-right: 1px solid #e5e7eb; padding-right: 8px; }
#   .right { display: flex; flex-direction: column; min-width: 0; }
#   .panel { position: relative; flex: 1 1 auto; display: flex; flex-direction: column; gap: 10px; }
#   .controls { position: relative; z-index: 2; background: #fff; display: flex; gap: 12px; align-items: center; flex-wrap: wrap; }
#   label { font-size: 0.9rem; color: #374151; }
#   .hint { color: #374151; font-size: 0.9rem; padding: 0 16px; }
#   .warn { color: #b45309; font-size: 0.85rem; padding: 4px 16px 0; min-height: 1.2em; }
#   table { border-collapse: collapse; width: 100%; }
#   td { padding: 5px; vertical-align: top; }
#   td:first-child { background-color: lightcyan; font: 0.8em sans-serif; white-space: nowrap; }
#   td:nth-child(2) { background-color: aliceblue; }
#   tr:nth-child(-n + 3) > td { background-color: transparent; }

#   #pdbeMolstar { position: relative; z-index: 1; width: 100%; height: 100%; overflow: hidden; border: 1px solid #e5e7eb; border-radius: 8px; }
#   #pdbeMolstar .msp-plugin,
#   #pdbeMolstar .msp-viewport { position: absolute; inset: 0; }

#   .legend { font-size: 12px; color: #374151; margin: 6px 0; }
# </style>

# <script type="importmap">
# {
#   "imports": {
#     "@nightingale-elements/": "https://cdn.jsdelivr.net/npm/@nightingale-elements/"
#   }
# }
# </script>

# <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/pdbe-molstar@3.8.0/build/pdbe-molstar.css">
# <script src="https://cdn.jsdelivr.net/npm/pdbe-molstar@3.8.0/build/pdbe-molstar-plugin.js"></script>

# <script id="features-cache"   type="application/json">__FEATURES_CACHE__</script>
# <script id="variation-cache"  type="application/json">__VARIATION_CACHE__</script>
# <script id="structures-cache" type="application/json">__STRUCTURES_CACHE__</script>
# <script id="scop3p-cache"     type="application/json">__SCOP3P_CACHE__</script>
# <script id="scop3p-compact"   type="application/json">__SCOP3P_COMPACT__</script>
# </head>

# <body>
# <div class="hint">1D left · 3D right (PDBe Mol*). Click in 1D to highlight; toggle Scop3P overlay. Base is uniform grey.</div>
# <div class="warn" id="warn"></div>

# <div class="grid">
#   <div class="left">
#     <nightingale-manager id="mgr">
#       <table>
#         <tbody>
#           <tr><td></td><td><nightingale-navigation id="navigation" min-width="800" height="40" length="770" display-start="1" display-end="770" margin-color="white" highlight-event="onclick"></nightingale-navigation></td></tr>
#           <tr><td></td><td><nightingale-sequence id="sequence" min-width="800" height="40" length="770" display-start="1" display-end="770" margin-color="white" highlight-event="onclick"></nightingale-sequence></td></tr>
#           <tr><td></td><td><nightingale-colored-sequence id="colored-sequence" min-width="800" height="15" length="770" display-start="1" display-end="770" scale="hydrophobicity-scale" margin-color="white" highlight-event="onclick"></nightingale-colored-sequence></td></tr>
#           <tr><td>Domain</td><td><nightingale-track id="domain" min-width="800" height="18" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick" display-labels="true"></nightingale-track><div id="domain-legend" class="legend"></div></td></tr>
#           <tr><td>Region</td><td><nightingale-track id="region" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
#           <tr><td>Site</td><td><nightingale-track id="site"   min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
#           <tr><td>Chain</td><td><nightingale-track id="chain"  layout="non-overlapping" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
#           <tr><td>Binding site</td><td><nightingale-track id="binding" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
#           <tr><td>Disulfide bond</td><td><nightingale-track id="disulfide-bond" layout="non-overlapping" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
#           <tr><td>Beta strand</td><td><nightingale-track id="beta-strand" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
#           <tr><td>Scop3P phosphorylation</td><td><nightingale-track id="scop3p" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
#           <tr><td>Variants</td><td><nightingale-linegraph-track id="variants" min-width="800" height="60" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-linegraph-track></td></tr>
#         </tbody>
#       </table>
#     </nightingale-manager>
#   </div>

#   <div class="right">
#     <div class="panel">
#       <div class="controls">
#         <label>Structure
#           <select id="structureSelect" style="margin-left:6px;padding:4px 6px;"></select>
#         </label>
#         <label style="margin-left:12px;">
#           <input type="checkbox" id="toggleScop3P" checked>
#           Show Scop3P sites
#         </label>
#       </div>
#       <div id="pdbeMolstar"></div>
#     </div>
#   </div>
# </div>

# <script type="module">
#   import "@nightingale-elements/nightingale-sequence@latest";
#   import "@nightingale-elements/nightingale-track@latest";
#   import "@nightingale-elements/nightingale-manager@latest";
#   import "@nightingale-elements/nightingale-navigation@latest";
#   import "@nightingale-elements/nightingale-colored-sequence@latest";
#   import "@nightingale-elements/nightingale-linegraph-track@latest";

#   const accession = "__ACC__";
#   const featuresCache   = JSON.parse(document.getElementById("features-cache").textContent || "{}");
#   const variationCache  = JSON.parse(document.getElementById("variation-cache").textContent || "{}");
#   const structuresCache = JSON.parse(document.getElementById("structures-cache").textContent || "{}");
#   const scop3pCache     = JSON.parse(document.getElementById("scop3p-cache").textContent || "{}");

#   const featuresData  = featuresCache[accession];
#   const variationData = variationCache[accession] || { features: [] };
#   const structures    = structuresCache[accession] || [{ id: `AF-${accession}-F1`, source:"AFDB", chain:null, label:`AlphaFold (AF-${accession}-F1)` }];

#   await Promise.all([
#     customElements.whenDefined("nightingale-sequence"),
#     customElements.whenDefined("nightingale-navigation"),
#     customElements.whenDefined("nightingale-manager"),
#     customElements.whenDefined("nightingale-track"),
#     customElements.whenDefined("nightingale-colored-sequence"),
#     customElements.whenDefined("nightingale-linegraph-track"),
#   ]);

#   const mgr = document.getElementById("mgr");
#   const navEl = document.getElementById("navigation");
#   const seqEl = document.getElementById("sequence");
#   const coloredEl = document.getElementById("colored-sequence");
#   const trDomain = document.getElementById("domain");
#   const domainLegend = document.getElementById("domain-legend");
#   const trRegion = document.getElementById("region");
#   const trSite = document.getElementById("site");
#   const trBinding = document.getElementById("binding");
#   const trChain = document.getElementById("chain");
#   const trDisulf = document.getElementById("disulfide-bond");
#   const trStrand = document.getElementById("beta-strand");
#   const trScop3P = document.getElementById("scop3p");
#   const variantsEl = document.getElementById("variants");
#   const structSelect = document.getElementById("structureSelect");
#   const toggleScop = document.getElementById("toggleScop3P");

#   const seq = featuresData.sequence;
#   const len = seq.length;
#   const setLen = (el) => { if (!el) return; el.length = len; el.setAttribute("length", String(len)); el.displayStart = 1; el.displayEnd = len; el.setAttribute("display-start","1"); el.setAttribute("display-end", String(len)); };
#   [navEl, seqEl, coloredEl, trDomain, trRegion, trSite, trBinding, trChain, trDisulf, trStrand, trScop3P, variantsEl].forEach(setLen);

#   seqEl.data = seq;
#   coloredEl.data = seq;

#   const feats = (featuresData.features || []).map((ft) => ({ ...ft, start: ft.start ?? ft.begin }));
#   trDomain.data = feats.filter(({ type }) => type === "DOMAIN").map(f => ({ ...f, label: f.description || f.type || "domain" }));
#   trRegion.data = feats.filter(({ type }) => type === "REGION");
#   trSite.data   = feats.filter(({ type }) => type === "SITE");
#   trBinding.data= feats.filter(({ type }) => type === "BINDING");
#   trChain.data  = feats.filter(({ type }) => type === "CHAIN");
#   trDisulf.data = feats.filter(({ type }) => type === "DISULFID");
#   trStrand.data = feats.filter(({ type }) => type === "STRAND");

#   try {
#     const items = trDomain.data.slice(0, 12).map(d => `${d.label || d.description || "domain"} (${d.start}-${d.end})`);
#     domainLegend.textContent = items.length ? "Domains: " + items.join(" · ") + (trDomain.data.length > 12 ? " …" : "") : "";
#   } catch(_) {}

#   const counts = new Map();
#   for (const v of (variationData.features || [])) {
#     const pos = Number(v.begin);
#     if (!Number.isFinite(pos)) continue;
#     counts.set(pos, (counts.get(pos) || 0) + 1);
#   }
#   const values = Array.from(counts, ([position, value]) => ({ position, value })).sort((a,b)=>a.position-b.position);
#   const max = values.length ? Math.max(...values.map(d=>d.value)) : 0;
#   variantsEl.data = [{ color: "grey", values, range: [0, Math.max(1, max)] }];

#   trScop3P.data = (scop3pCache[accession]?.modifications || [])
#     .filter(m => String(m.name).toLowerCase()==="phosphorylation")
#     .filter(m => Number.isFinite(Number(m.position)))
#     .map(m => ({ start: Number(m.position), end: Number(m.position), type: "SCOP3P_PHOSPHO", description: `Phospho ${(m.residue||'?').slice(0,1)}@${m.position} (${m.source||'Scop3P'})` }));

#   function populateStructureSelect(arr) {
#     structSelect.innerHTML = "";
#     // AF first
#     const afFirst = arr.sort((a,b) => (a.source === "AFDB" ? -1 : b.source === "AFDB" ? 1 : 0));
#     for (const s of afFirst) {
#       const opt = document.createElement("option");
#       opt.value = (s.source === "PDB" && s.chain) ? `${s.id}|${s.chain}` : s.id;
#       opt.textContent = s.label || (s.source === "PDB" ? `${s.id}${s.chain? " — chain "+s.chain : ""}` : s.id);
#       opt.dataset.source = s.source || "";
#       structSelect.appendChild(opt);
#     }
#     const afIdx = Array.from(structSelect.options).findIndex(o => o.value.startsWith(`AF-${accession}-F1`));
#     structSelect.selectedIndex = afIdx >= 0 ? afIdx : 0;
#   }
#   populateStructureSelect(structures);

#   // ---- PDBe Mol* (bounded & robust) ----
#   const PDBeMolstarPlugin = window.PDBeMolstarPlugin;
#   const viewer = new PDBeMolstarPlugin();

#   function getContainer() {
#     const el = document.getElementById("pdbeMolstar");
#     if (!el) throw new Error("#pdbeMolstar not found");
#     el.style.position = "relative";
#     el.style.width = "100%";
#     el.style.height = "100%";
#     return el;
#   }
#   function renderMolstar(options) {
#     const container = getContainer();
#     container.innerHTML = "";
#     requestAnimationFrame(() => viewer.render(container, options));
#   }

#   function buildScop3PSelection() {
#     const mods = (scop3pCache[accession]?.modifications || [])
#       .filter(m => String(m.name).toLowerCase() === "phosphorylation")
#       .filter(m => Number.isFinite(Number(m.position)));
#     return mods.map(m => ({
#       uniprot_accession: accession,
#       start_uniprot_residue_number: Number(m.position),
#       end_uniprot_residue_number: Number(m.position),
#       color: { r: 255, g: 0, b: 0 },
#       representation: "ball-and-stick",
#       sideChain: true
#     }));
#   }

#   function buildClickSelection(rangeStr) {
#     if (!rangeStr) return [];
#     return rangeStr.split(",").filter(Boolean).map(seg => {
#       const [a, b] = seg.split(":").map(Number);
#       return {
#         uniprot_accession: accession,
#         start_uniprot_residue_number: Math.min(a, b),
#         end_uniprot_residue_number: Math.max(a, b),
#         color: { r: 255, g: 221, b: 0 },
#         representation: "ball-and-stick",
#         sideChain: true
#       };
#     });
#   }

#   function applyOverlays() {
#     const navH = navEl.getAttribute("highlight") || "";
#     const clickData = buildClickSelection(navH);
#     const scopData = toggleScop.checked ? buildScop3PSelection() : [];
#     const combined = [...scopData, ...clickData];

#     const apply = () => viewer.visual.select({
#       data: combined,
#       nonSelectedColor: { r: 176, g: 176, b: 176 }
#     });

#     try { apply(); }
#     catch(_) { setTimeout(() => { try { apply(); } catch(_) {} }, 300); }
#   }

#   // Prefer PDBe lower-case .bcif; fall back to .cif if needed
#   async function choosePdbCustomData(pdbIdUpper) {
#     const id = pdbIdUpper.toLowerCase();
#     const bcif = `https://www.ebi.ac.uk/pdbe/entry-files/download/${id}.bcif`;
#     const cif  = `https://www.ebi.ac.uk/pdbe/entry-files/download/${id}.cif`;
#     try {
#       const r = await fetch(bcif, { method: "HEAD" });
#       if (r.ok) return { url: bcif, format: "bcif", binary: true };
#     } catch(_) {}
#     return { url: cif, format: "cif", binary: false };
#   }

#   async function loadSelectedStructure() {
#     const opt = structSelect.options[structSelect.selectedIndex];
#     const value = opt.value;
#     const source = opt.dataset.source || "";

#     let options;
#     if (source === "AFDB" || value.startsWith(`AF-${accession}-F1`)) {
#       const url = `https://alphafold.ebi.ac.uk/files/AF-${accession}-F1-model_v4.cif`;
#       options = {
#         customData: { url, format: "cif", binary: false },
#         hideControls: true,
#         bgColor: { r: 255, g: 255, b: 255 },
#         alphafoldView: false
#       };
#       renderMolstar(options);
#     } else {
#       const pdbId = value.split("|")[0]; // keep uppercase label, but fetch lower-case
#       const customData = await choosePdbCustomData(pdbId);
#       options = {
#         customData,
#         hideControls: true,
#         bgColor: { r: 255, g: 255, b: 255 }
#       };
#       renderMolstar(options);
#     }

#     const reassert = () => { applyOverlays(); };
#     if (viewer.events?.loadComplete?.subscribe) {
#       const sub = viewer.events.loadComplete.subscribe(() => {
#         reassert(); setTimeout(reassert, 200); setTimeout(reassert, 450);
#         try { sub.unsubscribe(); } catch(_) {}
#       });
#     } else {
#       setTimeout(reassert, 500);
#       setTimeout(reassert, 900);
#     }
#   }

#   // initial
#   loadSelectedStructure();

#   structSelect.addEventListener("change", () => { loadSelectedStructure(); });
#   toggleScop.addEventListener("change", () => applyOverlays());

#   const obs = new MutationObserver(() => applyOverlays());
#   obs.observe(navEl, { attributes: true, attributeFilter: ["highlight"] });
#   mgr.addEventListener("click", () => applyOverlays());
# </script>
# </body>
# </html>
# """

# def build_html(acc: str, out_dir: str = ".") -> Path:
#     b = build_bundle(acc)
#     features_cache   = json.dumps({acc: b["features"]}, ensure_ascii=False)
#     variation_cache  = json.dumps({acc: b["variation"]}, ensure_ascii=False)
#     structures_cache = json.dumps({acc: b["structures"]}, ensure_ascii=False)
#     scop3p_cache     = json.dumps({acc: b["scop3p"]}, ensure_ascii=False)
#     scop3p_compact   = json.dumps({acc: b["scop3p_compact"]}, ensure_ascii=False)

#     html_out = (
#         HTML_TEMPLATE
#         .replace("__ACC__", acc)
#         .replace("__FEATURES_CACHE__", features_cache)
#         .replace("__VARIATION_CACHE__", variation_cache)
#         .replace("__STRUCTURES_CACHE__", structures_cache)
#         .replace("__SCOP3P_CACHE__", scop3p_cache)
#         .replace("__SCOP3P_COMPACT__", scop3p_compact)
#     )
#     out_path = Path(out_dir) / f"nightingale_pdbe_{acc}.html"
#     out_path.write_text(html_out, encoding="utf-8")
#     return out_path

# # Example:
# # p = build_html("P05067"); p 


### Superpose option added here in cell 2

In [8]:
# Cell 2: 1D full-width (top) + two PDBe Mol* viewers (bottom: left mapped, right superposition)
# - Superposition toggle centered between top and bottom
# - 1D↔3D (left) stays interactive regardless of superposition state
# - Scop3P phospho sites highlighted as sticks on superposed structures within selected segment

from pathlib import Path
import json

HTML_TEMPLATE = """<!doctype html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width, initial-scale=1"/>
<title>Nightingale + PDBe Mol* — __ACC__</title>

<style>
  :root {
    --top-height: 34vh;          /* 1D block height (adjust if needed) */
    --gap: 16px;
    --panel-height: 560px;       /* left mapped viewer height */
    --superpanel-height: 420px;  /* right superposition viewer height */
  }
  html, body { height: auto !important; overflow: auto; }
  body { margin: 0; font-family: system-ui, -apple-system, Segoe UI, Roboto, sans-serif; }

  /* New grid: row1 = 1D, row2 = center toggle, row3 = two 3D panels */
  .grid {
    display: grid;
    grid-template-rows: var(--top-height) auto 1fr;
    grid-template-columns: 1fr;
    gap: var(--gap);
    padding: var(--gap);
    align-items: stretch;
  }

  .top-1d {
    background: #fff;
    border: 1px solid #e5e7eb;
    border-radius: 8px;
    overflow: auto;
    padding: 8px;
    min-height: 0;
  }

  .center-toggle {
    display: flex;
    justify-content: center;
    align-items: center;
    padding: 0 0 4px;
  }

  .bottom-3d {
    display: grid;
    grid-template-columns: 1fr 1fr;
    gap: var(--gap);
    min-height: 0;
  }

  .panel {
    position: relative;
    display: flex;
    flex-direction: column;
    gap: 10px;
    min-height: 0;
    background: #fff;
    border: 1px solid #e5e7eb;
    border-radius: 8px;
    padding: 10px;
  }

  .controls {
    position: relative;
    z-index: 2;
    background: #fff;
    display: flex;
    gap: 12px;
    align-items: center;
    flex-wrap: wrap;
  }

  .titlebar { display:flex; align-items:center; justify-content:space-between; }
  .titlebar h3 { margin: 0; font-size: 1rem; color:#111827; }
  .btn { padding: 6px 10px; border: 1px solid #e5e7eb; border-radius: 8px; background: #f9fafb; cursor: pointer; }
  .btn:hover { background:#f3f4f6; }
  label { font-size: 0.9rem; color: #374151; }
  .hint { color: #374151; font-size: 0.9rem; padding: 0 16px 8px; }
  .warn { color: #b45309; font-size: 0.85rem; padding: 4px 16px 8px; min-height: 1.2em; }

  table { border-collapse: collapse; width: 100%; }
  td { padding: 5px; vertical-align: top; }
  td:first-child { background-color: lightcyan; font: 0.8em sans-serif; white-space: nowrap; }
  td:nth-child(2) { background-color: aliceblue; }
  tr:nth-child(-n + 3) > td { background-color: transparent; }

  .viewer { position: relative; width: 100%; border: 1px solid #e5e7eb; border-radius: 8px; overflow: hidden; min-height: 0; }
  #pdbeMolstarMain  { height: var(--panel-height); min-height: var(--panel-height); }
  #pdbeMolstarSuper { height: var(--superpanel-height); min-height: var(--superpanel-height); display:none; }
  .viewer .msp-plugin, .viewer .msp-viewport { position:absolute; inset:0; }

  .legend { font-size: 12px; color: #374151; margin: 6px 0; }
  select { padding: 4px 6px; }
  input[type="color"] { width: 28px; height: 28px; padding: 0; border: 1px solid #e5e7eb; border-radius: 6px; }
  .flex-spacer { flex: 1; }
</style>

<script type="importmap">
{
  "imports": {
    "@nightingale-elements/": "https://cdn.jsdelivr.net/npm/@nightingale-elements/"
  }
}
</script>

<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/pdbe-molstar@3.8.0/build/pdbe-molstar.css">
<script src="https://cdn.jsdelivr.net/npm/pdbe-molstar@3.8.0/build/pdbe-molstar-plugin.js"></script>

<script id="features-cache"   type="application/json">__FEATURES_CACHE__</script>
<script id="variation-cache"  type="application/json">__VARIATION_CACHE__</script>
<script id="structures-cache" type="application/json">__STRUCTURES_CACHE__</script>
<script id="scop3p-cache"     type="application/json">__SCOP3P_CACHE__</script>
<script id="scop3p-compact"   type="application/json">__SCOP3P_COMPACT__</script>
</head>

<body>
<div class="hint">Top: 1D tracks. Bottom-left: Mapped structure (AF/PDB) with click + Scop3P overlays. Bottom-right: Superposition panel (PDBe-KB).</div>
<div class="warn" id="warn"></div>

<div class="grid">
  <!-- TOP: 1D full-width -->
  <div class="top-1d">
    <nightingale-manager id="mgr">
      <table>
        <tbody>
          <tr><td></td><td><nightingale-navigation id="navigation" min-width="800" height="40" length="770" display-start="1" display-end="770" margin-color="white" highlight-event="onclick"></nightingale-navigation></td></tr>
          <tr><td></td><td><nightingale-sequence id="sequence" min-width="800" height="40" length="770" display-start="1" display-end="770" margin-color="white" highlight-event="onclick"></nightingale-sequence></td></tr>
          <tr><td></td><td><nightingale-colored-sequence id="colored-sequence" min-width="800" height="15" length="770" display-start="1" display-end="770" scale="hydrophobicity-scale" margin-color="white" highlight-event="onclick"></nightingale-colored-sequence></td></tr>
          <tr><td>Domain</td><td><nightingale-track id="domain" min-width="800" height="18" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick" display-labels="true"></nightingale-track><div id="domain-legend" class="legend"></div></td></tr>
          <tr><td>Region</td><td><nightingale-track id="region" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Site</td><td><nightingale-track id="site"   min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Chain</td><td><nightingale-track id="chain"  layout="non-overlapping" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Binding site</td><td><nightingale-track id="binding" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Disulfide bond</td><td><nightingale-track id="disulfide-bond" layout="non-overlapping" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Beta strand</td><td><nightingale-track id="beta-strand" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Scop3P phosphorylation</td><td><nightingale-track id="scop3p" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Variants</td><td><nightingale-linegraph-track id="variants" min-width="800" height="60" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-linegraph-track></td></tr>
        </tbody>
      </table>
    </nightingale-manager>
  </div>

  <!-- CENTER: toggle to show/hide superposition panel -->
  <div class="center-toggle">
    <label>
      <input type="checkbox" id="toggleSuperpose" disabled>
      Show superposition (PDBe-KB)
    </label>
  </div>

  <!-- BOTTOM: two 3D panels -->
  <div class="bottom-3d">

    <!-- Bottom-left: Mapped structure -->
    <div class="panel">
      <div class="titlebar">
        <h3>Mapped structure</h3>
        <div>
          <button id="btnFitMain" class="btn" type="button">Fit</button>
        </div>
      </div>
      <div class="controls">
        <label>Structure
          <select id="structureSelect" style="margin-left:6px;"></select>
        </label>
        <label style="margin-left:12px;">
          <input type="checkbox" id="toggleScop3P" checked>
          Show Scop3P sites
        </label>
      </div>
      <div id="pdbeMolstarMain" class="viewer"></div>
    </div>

    <!-- Bottom-right: Superposition -->
    <div class="panel">
      <div class="titlebar">
        <h3>Superposition</h3>
        <div>
          <button id="btnPopOut" class="btn" type="button" title="Open in a new window">Pop-out</button>
          <button id="btnHideSuper" class="btn" type="button" title="Hide the superposition panel">Hide</button>
        </div>
      </div>

      <div class="controls">
        <label>Segment
          <select id="segmentSelect" disabled></select>
        </label>
        <label>Clusters
          <select id="clusterSelect" multiple size="3" disabled style="min-width:160px;"></select>
        </label>
        <label>
          <input type="checkbox" id="toggleCompleteCluster" disabled>
          Complete cluster
        </label>
        <label>
          <input type="checkbox" id="toggleLigandView" disabled>
          Ligand view
        </label>
        <label title="Ligand color">
          <input type="color" id="ligandColor" value="#ffff32" disabled>
        </label>

        <span class="flex-spacer"></span>

        <label title="Highlight Scop3P phospho sites (sticks) within the segment">
          <input type="checkbox" id="toggleScop3P_super" checked disabled>
          Scop3P sites
        </label>
        <span id="superHint" class="hint" style="padding:0 0 0 8px;"></span>
      </div>

      <div id="pdbeMolstarSuper" class="viewer"></div>
    </div>

  </div>
</div>

<script type="module">
  import "@nightingale-elements/nightingale-sequence@latest";
  import "@nightingale-elements/nightingale-track@latest";
  import "@nightingale-elements/nightingale-manager@latest";
  import "@nightingale-elements/nightingale-navigation@latest";
  import "@nightingale-elements/nightingale-colored-sequence@latest";
  import "@nightingale-elements/nightingale-linegraph-track@latest";

  const accession = "__ACC__";
  const featuresCache   = JSON.parse(document.getElementById("features-cache").textContent || "{}");
  const variationCache  = JSON.parse(document.getElementById("variation-cache").textContent || "{}");
  const structuresCache = JSON.parse(document.getElementById("structures-cache").textContent || "{}");
  const scop3pCache     = JSON.parse(document.getElementById("scop3p-cache").textContent || "{}");

  const featuresData  = featuresCache[accession];
  const variationData = variationCache[accession] || { features: [] };
  const structures    = structuresCache[accession] || [{ id: "AF-" + accession + "-F1", source:"AFDB", chain:null, label:"AlphaFold (AF-" + accession + "-F1)" }];

  await Promise.all([
    customElements.whenDefined("nightingale-sequence"),
    customElements.whenDefined("nightingale-navigation"),
    customElements.whenDefined("nightingale-manager"),
    customElements.whenDefined("nightingale-track"),
    customElements.whenDefined("nightingale-colored-sequence"),
    customElements.whenDefined("nightingale-linegraph-track")
  ]);

  const warnEl = document.getElementById("warn");
  const mgr = document.getElementById("mgr");
  const navEl = document.getElementById("navigation");
  const seqEl = document.getElementById("sequence");
  const coloredEl = document.getElementById("colored-sequence");
  const trDomain = document.getElementById("domain");
  const domainLegend = document.getElementById("domain-legend");
  const trRegion = document.getElementById("region");
  const trSite = document.getElementById("site");
  const trBinding = document.getElementById("binding");
  const trChain = document.getElementById("chain");
  const trDisulf = document.getElementById("disulfide-bond");
  const trStrand = document.getElementById("beta-strand");
  const trScop3P = document.getElementById("scop3p");
  const variantsEl = document.getElementById("variants");

  const structSelect = document.getElementById("structureSelect");
  const toggleScop = document.getElementById("toggleScop3P");

  const toggleSuper = document.getElementById("toggleSuperpose");
  const segSel = document.getElementById("segmentSelect");
  const cluSel = document.getElementById("clusterSelect");
  const chkComplete = document.getElementById("toggleCompleteCluster");
  const chkLigand = document.getElementById("toggleLigandView");
  const ligandColor = document.getElementById("ligandColor");
  const toggleScopSuper = document.getElementById("toggleScop3P_super");
  const superHint = document.getElementById("superHint");

  const mainDiv  = document.getElementById("pdbeMolstarMain");
  const superDiv = document.getElementById("pdbeMolstarSuper");
  const btnPopOut = document.getElementById("btnPopOut");
  const btnHideSuper = document.getElementById("btnHideSuper");
  const btnFitMain = document.getElementById("btnFitMain");

  let segmentsMeta = [];

  // 1D lengths
  const seq = featuresData.sequence;
  const len = seq.length;
  const setLen = (el) => { if (!el) return; el.length = len; el.setAttribute("length", String(len)); el.displayStart = 1; el.displayEnd = len; el.setAttribute("display-start","1"); el.setAttribute("display-end", String(len)); };
  [navEl, seqEl, coloredEl, trDomain, trRegion, trSite, trBinding, trChain, trDisulf, trStrand, trScop3P, variantsEl].forEach(setLen);

  seqEl.data = seq;
  coloredEl.data = seq;

  const feats = (featuresData.features || []).map(ft => ({ ...ft, start: ft.start ?? ft.begin }));
  trDomain.data = feats.filter(f => f.type === "DOMAIN").map(f => ({ ...f, label: f.description || f.type || "domain" }));
  trRegion.data = feats.filter(f => f.type === "REGION");
  trSite.data   = feats.filter(f => f.type === "SITE");
  trBinding.data= feats.filter(f => f.type === "BINDING");
  trChain.data  = feats.filter(f => f.type === "CHAIN");
  trDisulf.data = feats.filter(f => f.type === "DISULFID");
  trStrand.data = feats.filter(f => f.type === "STRAND");

  try {
    const items = trDomain.data.slice(0, 12).map(d => `${d.label || d.description || "domain"} (${d.start}-${d.end})`);
    domainLegend.textContent = items.length ? "Domains: " + items.join(" · ") + (trDomain.data.length > 12 ? " …" : "") : "";
  } catch(e) {}

  const counts = new Map();
  for (const v of (variationData.features || [])) {
    const pos = Number(v.begin);
    if (!Number.isFinite(pos)) continue;
    counts.set(pos, (counts.get(pos) || 0) + 1);
  }
  const values = Array.from(counts, ([position, value]) => ({ position, value })).sort((a,b)=>a.position-b.position);
  const max = values.length ? Math.max(...values.map(d=>d.value)) : 0;
  variantsEl.data = [{ color: "grey", values, range: [0, Math.max(1, max)] }];

  trScop3P.data = (scop3pCache[accession]?.modifications || [])
    .filter(m => String(m.name).toLowerCase()==="phosphorylation")
    .filter(m => Number.isFinite(Number(m.position)))
    .map(m => ({ start: +m.position, end: +m.position, type: "SCOP3P_PHOSPHO", description: `Phospho ${(m.residue||'?').slice(0,1)}@${m.position} (${m.source||'Scop3P'})` }));

  // Populate structures
  function populateStructureSelect(arr) {
    structSelect.innerHTML = "";
    const afFirst = arr.slice().sort((a,b) => (a.source === "AFDB" ? -1 : (b.source === "AFDB" ? 1 : 0)));
    for (const s of afFirst) {
      const opt = document.createElement("option");
      opt.value = (s.source === "PDB" && s.chain) ? `${s.id}|${s.chain}` : s.id;
      opt.textContent = s.label || (s.source === "PDB" ? `${s.id}${s.chain? " — chain "+s.chain : ""}` : s.id);
      opt.dataset.source = s.source || "";
      structSelect.appendChild(opt);
    }
    const afIdx = Array.from(structSelect.options).findIndex(o => o.value.startsWith(`AF-${accession}-F1`));
    structSelect.selectedIndex = afIdx >= 0 ? afIdx : 0;
  }
  populateStructureSelect(structures);

  // ===== Mol* helpers: two viewers + safe render =====
  const PDBeMolstarPlugin = window.PDBeMolstarPlugin;
  let viewerMain = null;
  let viewerSuper = null;

  function waitForNonZeroSize(el) {
    return new Promise((resolve) => {
      let ok = 0;
      const check = () => {
        const r = el.getBoundingClientRect();
        if (r.width > 0 && r.height > 0) { ok++; if (ok >= 2) return resolve(); }
        else ok = 0;
        requestAnimationFrame(check);
      };
      const ro = new ResizeObserver(() => {
        const r = el.getBoundingClientRect();
        if (r.width > 0 && r.height > 0) { ok++; if (ok >= 2) { ro.disconnect(); resolve(); } }
      });
      ro.observe(el);
      check();
    });
  }

  async function renderInto(container, options, which) {
    container.innerHTML = "";
    await waitForNonZeroSize(container);
    try {
      const v = which === "main" ? viewerMain : viewerSuper;
      if (v?.plugin?.dispose) v.plugin.dispose();
    } catch {}
    const inst = new PDBeMolstarPlugin();
    inst.render(container, options);
    if (which === "main") viewerMain = inst; else viewerSuper = inst;
    return inst;
  }

  // ===== Overlays (MAIN viewer) =====
  function buildScop3PSelection() {
    const mods = (scop3pCache[accession]?.modifications || [])
      .filter(m => String(m.name).toLowerCase() === "phosphorylation")
      .filter(m => Number.isFinite(Number(m.position)));
    return mods.map(m => ({
      uniprot_accession: accession,
      start_uniprot_residue_number: +m.position,
      end_uniprot_residue_number: +m.position,
      color: { r: 255, g: 0, b: 0 },
      representation: "ball-and-stick",
      sideChain: true
    }));
  }
  function buildClickSelection(rangeStr) {
    if (!rangeStr) return [];
    return rangeStr.split(",").filter(Boolean).map(seg => {
      const [a,b] = seg.split(":").map(Number);
      return {
        uniprot_accession: accession,
        start_uniprot_residue_number: Math.min(a,b),
        end_uniprot_residue_number: Math.max(a,b),
        color: { r: 255, g: 221, b: 0 },
        representation: "ball-and-stick",
        sideChain: true
      };
    });
  }
  function applyOverlays() {
    const v = viewerMain;
    if (!v?.visual?.select) return;
    const navH = navEl.getAttribute("highlight") || "";
    const clickData = buildClickSelection(navH);
    const scopData = toggleScop.checked ? buildScop3PSelection() : [];
    const combined = [...scopData, ...clickData];
    const apply = () => v.visual.select({ data: combined, nonSelectedColor: { r: 176, g: 176, b: 176 } });
    try { apply(); } catch { setTimeout(() => { try { apply(); } catch {} }, 300); }
  }

  // ===== Data helpers =====
  async function choosePdbCustomData(pdbIdUpper) {
    const id = pdbIdUpper.toLowerCase();
    const bcif = `https://www.ebi.ac.uk/pdbe/entry-files/download/${id}.bcif`;
    const cif  = `https://www.ebi.ac.uk/pdbe/entry-files/download/${id}.cif`;
    try {
      const r = await fetch(bcif, { method: "HEAD" });
      if (r && r.ok) return { url: bcif, format: "bcif", binary: true };
    } catch(_) {}
    return { url: cif, format: "cif", binary: false };
  }

  // ===== Main viewer loader =====
  async function loadMain() {
    const opt = structSelect.options[structSelect.selectedIndex];
    const value = opt ? opt.value : "";
    const source = opt ? (opt.dataset.source || "") : "";

    if (source === "AFDB" || value.startsWith(`AF-${accession}-F1`)) {
      const url = `https://alphafold.ebi.ac.uk/files/AF-${accession}-F1-model_v6.cif`;
      await renderInto(mainDiv, {
        customData: { url, format: "cif", binary: false },
        hideControls: true,
        bgColor: { r: 255, g: 255, b: 255 },
        alphafoldView: false
      }, "main");
    } else {
      const pdbId = value.split("|")[0] || "";
      if (!pdbId) return;
      const customData = await choosePdbCustomData(pdbId);
      await renderInto(mainDiv, {
        customData,
        hideControls: true,
        bgColor: { r: 255, g: 255, b: 255 }
      }, "main");
    }

    const reassert = () => applyOverlays();
    if (viewerMain?.events?.loadComplete?.subscribe) {
      const sub = viewerMain.events.loadComplete.subscribe(() => {
        reassert(); setTimeout(reassert, 200); setTimeout(reassert, 450);
        try { sub.unsubscribe(); } catch {}
      });
    } else {
      setTimeout(reassert, 500);
      setTimeout(reassert, 900);
    }
  }

  // ===== PDBe Graph API for superposition =====
  async function fetchSuperpositionGraph(uniprot) {
    try {
      const url = "https://www.ebi.ac.uk/pdbe/graph-api/uniprot/superposition/" + uniprot;
      const res = await fetch(url, { cache: "no-store" });
      if (!res || !res.ok) return [];
      const js = await res.json();
      const arr = js && js[uniprot] ? js[uniprot] : [];
      if (!Array.isArray(arr)) return [];
      const out = [];
      for (let i=0; i<arr.length; i++) {
        const seg = arr[i];
        const segId = i + 1; // 1-based
        const clusters = [];
        const raws = Array.isArray(seg.clusters) ? seg.clusters : [];
        for (let k=0; k<raws.length; k++) {
          const list = raws[k];
          clusters.push({ id: k + 1, size: Array.isArray(list) ? list.length : 0,
                          repr: Array.isArray(list) ? list.find(x => x?.is_representative) : null });
        }
        out.push({ id: segId, start: +seg.segment_start, end: +seg.segment_end, clusters });
      }
      return out;
    } catch { return []; }
  }

  function populateSegmentsUI(segments) {
    segSel.innerHTML = "";
    for (const s of segments) {
      const opt = document.createElement("option");
      opt.value = String(s.id);
      opt.textContent = `Segment ${s.id} (${s.start}-${s.end})`;
      segSel.appendChild(opt);
    }
    segSel.disabled = segments.length === 0;
  }
  function populateClustersUI(segments, segId) {
    const s = segments.find(x => x.id == segId);
    const cls = (s && s.clusters) ? s.clusters : [];
    cluSel.innerHTML = "";
    for (const c of cls) {
      const opt = document.createElement("option");
      const reprTxt = (c.repr && c.repr.pdb_id) ? ` repr ${c.repr.pdb_id}:${(c.repr.auth_asym_id || c.repr.struct_asym_id || "")}` : "";
      opt.value = String(c.id);
      opt.textContent = `Cluster ${c.id} (n=${c.size})${reprTxt}`;
      cluSel.appendChild(opt);
    }
    cluSel.disabled = cls.length === 0;
  }

  function buildSuperpositionOptions({ matrixAccession, segment, clusters, completeCluster, ligandView, ligandColor, superposeAll }) {
    const seg = Number(segment);
    if (!Number.isFinite(seg) || seg < 1) throw new Error("Invalid segment index");

    const opts = {
      moleculeId: String(matrixAccession || accession).toUpperCase(),
      hideControls: true,
      bgColor: { r: 255, g: 255, b: 255 },
      superposition: true,
      superpositionParams: {
        matrixAccession: String(matrixAccession || "").toUpperCase(),
        segment: seg,
        superposeAll: !!superposeAll
      }
    };

    const cls = Array.isArray(clusters) ? clusters.map(n => Number(n)).filter(n => Number.isFinite(n) && n >= 1) : [];
    if (cls.length) opts.superpositionParams.cluster = cls;
    if (completeCluster != null) opts.superpositionParams.superposeCompleteCluster = !!completeCluster;
    if (ligandView) {
      opts.superpositionParams.ligandView = true;
      if (ligandColor && typeof ligandColor === "string" && /^#?[0-9a-f]{6}$/i.test(ligandColor)) {
        opts.superpositionParams.ligandColor = ligandColor.startsWith("#") ? ligandColor : ("#" + ligandColor);
      }
    }
    return opts;
  }

  // ===== Scop3P overlays on SUPERPOSITION viewer =====
  function buildScop3PSelectionInRange(start, end) {
    const mods = (scop3pCache[accession]?.modifications || [])
      .filter(m => String(m.name).toLowerCase() === "phosphorylation")
      .map(m => ({ pos: +m.position, residue: (m.residue || "?")[0] }))
      .filter(m => Number.isFinite(m.pos) && m.pos >= start && m.pos <= end);

    return mods.map(m => ({
      uniprot_accession: accession,
      start_uniprot_residue_number: m.pos,
      end_uniprot_residue_number: m.pos,
      color: { r: 255, g: 0, b: 0 },
      representation: "ball-and-stick",
      sideChain: true
    }));
  }

  function applySuperOverlays(segId) {
    if (!viewerSuper?.visual?.select) return;
    const seg = segmentsMeta.find(s => s.id == segId);
    if (!seg) return;

    const data = (toggleScopSuper?.checked)
      ? buildScop3PSelectionInRange(seg.start, seg.end)
      : [];

    const apply = () => viewerSuper.visual.select({
      data,
      nonSelectedColor: { r: 176, g: 176, b: 176 }
    });

    try { apply(); }
    catch { setTimeout(() => { try { apply(); } catch {} }, 300); }
  }

  async function renderSuperposition(segments) {
    [segSel, cluSel, chkComplete, chkLigand, ligandColor, toggleScopSuper].forEach(el => el.disabled = false);

    let segId = parseInt(segSel.value, 10);
    if (!Number.isFinite(segId) && segments.length) {
      segId = segments[0].id; segSel.value = String(segId);
    }
    if (!Number.isFinite(segId)) {
      warnEl.textContent = "No valid segment available for superposition.";
      toggleSuper.checked = false;
      superDiv.style.display = "none";
      try { viewerSuper?.plugin?.dispose?.(); } catch {}
      viewerSuper = null;
      return;
    }

    const clusters = Array.from(cluSel.selectedOptions).map(o => parseInt(o.value, 10)).filter(Number.isFinite);
    const col = ligandColor.value || "#ffff32";

    const options = buildSuperpositionOptions({
      matrixAccession: accession,
      segment: segId,
      clusters,
      completeCluster: chkComplete.checked,
      ligandView: chkLigand.checked,
      ligandColor: col,
      superposeAll: clusters.length === 0
    });

    superDiv.style.display = "block";
    await renderInto(superDiv, options, "super");
    superHint.textContent = clusters.length ? `Segment ${segId}, clusters [${clusters.join(", ")}]` : `Segment ${segId} (all)`;

    // Apply Scop3P overlay to superposed structures, including on loadComplete
    applySuperOverlays(segId);
    if (viewerSuper?.events?.loadComplete?.subscribe) {
      const sub2 = viewerSuper.events.loadComplete.subscribe(() => {
        applySuperOverlays(segId);
        try { sub2.unsubscribe(); } catch {}
      });
    }
  }

  // ===== Events =====
  structSelect.addEventListener("change", loadMain);
  toggleScop.addEventListener("change", () => applyOverlays());

  const obs = new MutationObserver(() => applyOverlays());
  obs.observe(navEl, { attributes: true, attributeFilter: ["highlight"] });
  mgr.addEventListener("click", () => applyOverlays());

  btnFitMain.addEventListener("click", () => { try { viewerMain?.visual?.reset({ camera: true }); } catch {} });

  btnHideSuper.addEventListener("click", () => {
    toggleSuper.checked = false;
    superDiv.style.display = "none";
    superHint.textContent = "";
    try { viewerSuper?.plugin?.dispose?.(); } catch {}
    viewerSuper = null;
  });

  btnPopOut.addEventListener("click", () => {
    if (!toggleSuper.checked) return;
    const segId = segSel.value || "1";
    const clusters = Array.from(cluSel.selectedOptions).map(o => o.value);
    const params = {
      matrixAccession: accession,
      segment: segId,
      cluster: clusters.join(","),
      complete: chkComplete.checked ? "1" : "0",
      ligand: chkLigand.checked ? "1" : "0",
      color: ligandColor.value.replace("#","")
    };
    const q = new URLSearchParams(params).toString();
    const w = window.open("", "_blank", "width=960,height=720");
    if (!w) return;

    // --- Safe string concatenation (no back-ticks) ---
    const popupHtml =
      '<!doctype html><meta charset="utf-8">' +
      '<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/pdbe-molstar@3.8.0/build/pdbe-molstar.css">' +
      '<script src="https://cdn.jsdelivr.net/npm/pdbe-molstar@3.8.0/build/pdbe-molstar-plugin.js"><\\/script>' +
      '<body style="margin:0">' +
      '<div id="v" style="position:fixed;inset:0"></div>' +
      '<script>' +
      '(function(){' +
        'var p=new URLSearchParams("' + q + '");' +
        'var acc=p.get("matrixAccession");' +
        'var seg=Number(p.get("segment"));' +
        'var cls=(p.get("cluster")||"").split(",").map(Number).filter(function(n){return Number.isFinite(n)&&n>0});' +
        'var complete=p.get("complete")==="1";' +
        'var ligand=p.get("ligand")==="1";' +
        'var color=p.get("color");' +
        'var opts={moleculeId: acc, hideControls:true,bgColor:{r:255,g:255,b:255},superposition:true,superpositionParams:{matrixAccession:acc,segment:seg,superposeAll:!cls.length}};' +
        'if(cls.length)opts.superpositionParams.cluster=cls;' +
        'if(complete)opts.superpositionParams.superposeCompleteCluster=true;' +
        'if(ligand){opts.superpositionParams.ligandView=true;if(color)opts.superpositionParams.ligandColor="#"+color;}' +
        'var el=document.getElementById("v");' +
        'function wait(){var r=el.getBoundingClientRect();if(r.width>0&&r.height>0){new window.PDBeMolstarPlugin().render(el,opts);}else{requestAnimationFrame(wait);}}' +
        'wait();' +
      '})();' +
      '<\\/script>' +
      '</body>';

    w.document.open();
    w.document.write(popupHtml);
    w.document.close();
  });

  toggleSuper.addEventListener("change", () => {
    if (toggleSuper.checked) {
      renderSuperposition(segmentsMeta);
      superDiv.style.display = "block";
    } else {
      btnHideSuper.click();
    }
  });

  toggleScopSuper.addEventListener("change", () => {
    if (toggleSuper.checked) applySuperOverlays(segSel.value);
  });

  segSel.addEventListener("change", () => {
    populateClustersUI(segmentsMeta, segSel.value);
    if (toggleSuper.checked) renderSuperposition(segmentsMeta);
  });
  cluSel.addEventListener("change", () => { if (toggleSuper.checked) renderSuperposition(segmentsMeta); });
  chkComplete.addEventListener("change", () => { if (toggleSuper.checked) renderSuperposition(segmentsMeta); });
  chkLigand.addEventListener("change", () => { if (toggleSuper.checked) renderSuperposition(segmentsMeta); });
  ligandColor.addEventListener("input", () => { if (toggleSuper.checked) renderSuperposition(segmentsMeta); });

  // ===== Init: main viewer + superposition metadata =====
  await loadMain();

  const segments = await fetchSuperpositionGraph(accession);
  segmentsMeta = segments;

  if (!segments.length) {
    toggleSuper.disabled = true;
    [segSel, cluSel, chkComplete, chkLigand, ligandColor, toggleScopSuper].forEach(el => el.disabled = true);
    warnEl.textContent = "Superposition not available for " + accession + " (no PDBe-KB segments).";
  } else {
    toggleSuper.disabled = false;
    warnEl.textContent = "";
    populateSegmentsUI(segments);
    segSel.value = String(segments[0].id);
    populateClustersUI(segments, segSel.value);
    // Super controls stay disabled until user toggles on; they enable in renderSuperposition()
  }
</script>
</body>
</html>
"""

def build_html(acc: str, out_dir: str = ".") -> Path:
    b = build_bundle(acc)
    features_cache   = json.dumps({acc: b["features"]}, ensure_ascii=False)
    variation_cache  = json.dumps({acc: b["variation"]}, ensure_ascii=False)
    structures_cache = json.dumps({acc: b["structures"]}, ensure_ascii=False)
    scop3p_cache     = json.dumps({acc: b["scop3p"]}, ensure_ascii=False)
    scop3p_compact   = json.dumps({acc: b["scop3p_compact"]}, ensure_ascii=False)

    html_out = (
        HTML_TEMPLATE
        .replace("__ACC__", acc)
        .replace("__FEATURES_CACHE__", features_cache)
        .replace("__VARIATION_CACHE__", variation_cache)
        .replace("__STRUCTURES_CACHE__", structures_cache)
        .replace("__SCOP3P_CACHE__", scop3p_cache)
        .replace("__SCOP3P_COMPACT__", scop3p_compact)
    )
    out_path = Path(out_dir) / f"nightingale_pdbe_backbone{acc}.html"
    out_path.write_text(html_out, encoding="utf-8")
    return out_path

# Example:
# p = build_html("P07949"); p


In [14]:
p = build_html("P07949")
p


PosixPath('nightingale_pdbe_backboneP07949.html')

### Superpose but not default layout -> switch or force to cartoon!


In [1]:
# # Cell 2: 1D full-width (top) + two viewers (bottom)
# # - Left: PDBe Mol* (unchanged)
# # - Right: Mol* core SUPERPOSITION with full control
# #   • Base = uniform grey CARTOON
# #   • Scop3P sites = red ball-and-stick
# #   • Uses PDBe Graph API for segments/clusters and PDBe mappings (SIFTS) to map UniProt→PDB residues
# #   • Aligns structures with Kabsch on CA atoms within the selected segment
# #
# # IMPORTANT: No top-level 'await' in <script>. All awaited code lives inside async main().

# from pathlib import Path
# import json

# HTML_TEMPLATE = """<!doctype html>
# <html lang="en">
# <head>
# <meta charset="utf-8"/>
# <meta name="viewport" content="width=device-width, initial-scale=1"/>
# <title>Nightingale + Mol* Superposition — __ACC__</title>

# <style>
#   :root {
#     --top-height: 34vh;
#     --gap: 16px;
#     --panel-height: 560px;
#     --superpanel-height: 420px;
#   }
#   html, body { height: auto !important; overflow: auto; }
#   body { margin: 0; font-family: system-ui, -apple-system, Segoe UI, Roboto, sans-serif; }

#   .grid {
#     display: grid;
#     grid-template-rows: var(--top-height) auto 1fr;
#     grid-template-columns: 1fr;
#     gap: var(--gap);
#     padding: var(--gap);
#     align-items: stretch;
#   }

#   .top-1d {
#     background: #fff;
#     border: 1px solid #e5e7eb;
#     border-radius: 8px;
#     overflow: auto;
#     padding: 8px;
#     min-height: 0;
#   }

#   .center-toggle {
#     display: flex;
#     justify-content: center;
#     align-items: center;
#     padding: 0 0 4px;
#   }

#   .bottom-3d {
#     display: grid;
#     grid-template-columns: 1fr 1fr;
#     gap: var(--gap);
#     min-height: 0;
#   }

#   .panel {
#     position: relative;
#     display: flex;
#     flex-direction: column;
#     gap: 10px;
#     min-height: 0;
#     background: #fff;
#     border: 1px solid #e5e7eb;
#     border-radius: 8px;
#     padding: 10px;
#   }

#   .controls {
#     position: relative;
#     z-index: 2;
#     background: #fff;
#     display: flex;
#     gap: 12px;
#     align-items: center;
#     flex-wrap: wrap;
#   }

#   .titlebar { display:flex; align-items:center; justify-content:space-between; }
#   .titlebar h3 { margin: 0; font-size: 1rem; color:#111827; }
#   .btn { padding: 6px 10px; border: 1px solid #e5e7eb; border-radius: 8px; background: #f9fafb; cursor: pointer; }
#   .btn:hover { background:#f3f4f6; }
#   label { font-size: 0.9rem; color: #374151; }
#   .hint { color: #374151; font-size: 0.9rem; padding: 0 16px 8px; }
#   .warn { color: #b45309; font-size: 0.85rem; padding: 4px 16px 8px; min-height: 1.2em; }

#   table { border-collapse: collapse; width: 100%; }
#   td { padding: 5px; vertical-align: top; }
#   td:first-child { background-color: lightcyan; font: 0.8em sans-serif; white-space: nowrap; }
#   td:nth-child(2) { background-color: aliceblue; }
#   tr:nth-child(-n + 3) > td { background-color: transparent; }

#   .viewer { position: relative; width: 100%; border: 1px solid #e5e7eb; border-radius: 8px; overflow: hidden; min-height: 0; }
#   #pdbeMolstarMain  { height: var(--panel-height); min-height: var(--panel-height); }
#   #molstarSuper     { height: var(--superpanel-height); min-height: var(--superpanel-height); display:none; }
#   .viewer .msp-plugin, .viewer .msp-viewport { position:absolute; inset:0; }

#   .legend { font-size: 12px; color: #374151; margin: 6px 0; }
#   select { padding: 4px 6px; }
#   input[type="color"] { width: 28px; height: 28px; padding: 0; border: 1px solid #e5e7eb; border-radius: 6px; }
#   .flex-spacer { flex: 1; }

#   .chip { font-size:.8rem; padding:2px 6px; border:1px solid #e5e7eb; border-radius:6px; background:#fafafa; }
# </style>

# <script type="importmap">
# {
#   "imports": {
#     "@nightingale-elements/": "https://cdn.jsdelivr.net/npm/@nightingale-elements/"
#   }
# }
# </script>

# <!-- Left panel (PDBe wrapper) -->
# <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/pdbe-molstar@3.8.0/build/pdbe-molstar.css">
# <script src="https://cdn.jsdelivr.net/npm/pdbe-molstar@3.8.0/build/pdbe-molstar-plugin.js"></script>

# <!-- Right panel (Mol* core) -->
# <link rel="stylesheet" href="https://unpkg.com/molstar/build/viewer/molstar.css">
# <script src="https://unpkg.com/molstar/build/viewer/molstar.js"></script>

# <script id="features-cache"   type="application/json">__FEATURES_CACHE__</script>
# <script id="variation-cache"  type="application/json">__VARIATION_CACHE__</script>
# <script id="structures-cache" type="application/json">__STRUCTURES_CACHE__</script>
# <script id="scop3p-cache"     type="application/json">__SCOP3P_CACHE__</script>
# <script id="scop3p-compact"   type="application/json">__SCOP3P_COMPACT__</script>
# </head>

# <body>
# <div class="hint">
#   Top: 1D tracks. Bottom-left: PDBe viewer (mapped structure). Bottom-right: <span class="chip">Mol* core superposition</span> (grey cartoon + red Scop3P sticks).
# </div>
# <div class="warn" id="warn"></div>

# <div class="grid">
#   <!-- TOP: 1D full-width -->
#   <div class="top-1d">
#     <nightingale-manager id="mgr">
#       <table>
#         <tbody>
#           <tr><td></td><td><nightingale-navigation id="navigation" min-width="800" height="40" length="770" display-start="1" display-end="770" margin-color="white" highlight-event="onclick"></nightingale-navigation></td></tr>
#           <tr><td></td><td><nightingale-sequence id="sequence" min-width="800" height="40" length="770" display-start="1" display-end="770" margin-color="white" highlight-event="onclick"></nightingale-sequence></td></tr>
#           <tr><td></td><td><nightingale-colored-sequence id="colored-sequence" min-width="800" height="15" length="770" display-start="1" display-end="770" scale="hydrophobicity-scale" margin-color="white" highlight-event="onclick"></nightingale-colored-sequence></td></tr>
#           <tr><td>Domain</td><td><nightingale-track id="domain" min-width="800" height="18" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick" display-labels="true"></nightingale-track><div id="domain-legend" class="legend"></div></td></tr>
#           <tr><td>Region</td><td><nightingale-track id="region" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
#           <tr><td>Site</td><td><nightingale-track id="site"   min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
#           <tr><td>Chain</td><td><nightingale-track id="chain"  layout="non-overlapping" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
#           <tr><td>Binding site</td><td><nightingale-track id="binding" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
#           <tr><td>Disulfide bond</td><td><nightingale-track id="disulfide-bond" layout="non-overlapping" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
#           <tr><td>Beta strand</td><td><nightingale-track id="beta-strand" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
#           <tr><td>Scop3P phosphorylation</td><td><nightingale-track id="scop3p" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
#           <tr><td>Variants</td><td><nightingale-linegraph-track id="variants" min-width="800" height="60" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-linegraph-track></td></tr>
#         </tbody>
#       </table>
#     </nightingale-manager>
#   </div>

#   <!-- CENTER: toggle to show/hide superposition panel -->
#   <div class="center-toggle">
#     <label>
#       <input type="checkbox" id="toggleSuperpose" disabled>
#       Show superposition (Mol* core)
#     </label>
#   </div>

#   <!-- BOTTOM: two 3D panels -->
#   <div class="bottom-3d">

#     <!-- Bottom-left: Mapped structure (PDBe wrapper, unchanged) -->
#     <div class="panel">
#       <div class="titlebar">
#         <h3>Mapped structure</h3>
#         <div><button id="btnFitMain" class="btn" type="button">Fit</button></div>
#       </div>
#       <div class="controls">
#         <label>Structure
#           <select id="structureSelect" style="margin-left:6px;"></select>
#         </label>
#         <label style="margin-left:12px;">
#           <input type="checkbox" id="toggleScop3P" checked>
#           Show Scop3P sites
#         </label>
#       </div>
#       <div id="pdbeMolstarMain" class="viewer"></div>
#     </div>

#     <!-- Bottom-right: Superposition (Mol* core) -->
#     <div class="panel">
#       <div class="titlebar">
#         <h3>Superposition (Mol* core)</h3>
#         <div>
#           <span id="superInfo" class="hint"></span>
#           <button id="btnFitSuper" class="btn" type="button" title="Fit">Fit</button>
#           <button id="btnHideSuper" class="btn" type="button" title="Hide the superposition panel">Hide</button>
#         </div>
#       </div>

#       <div class="controls">
#         <label>Segment
#           <select id="segmentSelect" disabled></select>
#         </label>
#         <label>Clusters
#           <select id="clusterSelect" multiple size="3" disabled style="min-width:180px;"></select>
#         </label>
#         <label>
#           <input type="checkbox" id="toggleCompleteCluster" disabled>
#           Complete cluster
#         </label>
#         <label title="(placeholder in Mol* core)">
#           <input type="checkbox" id="toggleLigandView" disabled>
#           Ligand view
#         </label>
#         <label title="Ligand color (placeholder)">
#           <input type="color" id="ligandColor" value="#ffff32" disabled>
#         </label>

#         <span class="flex-spacer"></span>

#         <label title="Highlight Scop3P phospho sites (sticks) within the segment">
#           <input type="checkbox" id="toggleScop3P_super" checked disabled>
#           Scop3P sites
#         </label>
#         <span id="superHint" class="hint" style="padding:0 0 0 8px;"></span>
#       </div>

#       <div id="molstarSuper" class="viewer"></div>
#     </div>

#   </div>
# </div>

# <script type="module">
#   /* 1) Imports must stay top-level in a module */
#   import "@nightingale-elements/nightingale-sequence@latest";
#   import "@nightingale-elements/nightingale-track@latest";
#   import "@nightingale-elements/nightingale-manager@latest";
#   import "@nightingale-elements/nightingale-navigation@latest";
#   import "@nightingale-elements/nightingale-colored-sequence@latest";
#   import "@nightingale-elements/nightingale-linegraph-track@latest";

#   /* 2) Everything else lives inside async main() (no top-level 'await' anywhere) */
#   async function main() {
#     const accession = "__ACC__";
#     const featuresCache   = JSON.parse(document.getElementById("features-cache").textContent || "{}");
#     const variationCache  = JSON.parse(document.getElementById("variation-cache").textContent || "{}");
#     const structuresCache = JSON.parse(document.getElementById("structures-cache").textContent || "{}");
#     const scop3pCache     = JSON.parse(document.getElementById("scop3p-cache").textContent || "{}");

#     const featuresData  = featuresCache[accession];
#     const variationData = variationCache[accession] || { features: [] };
#     const structures    = structuresCache[accession] || [{ id: "AF-" + accession + "-F1", source:"AFDB", chain:null, label:"AlphaFold (AF-" + accession + "-F1)" }];

#     const warnEl = document.getElementById("warn");
#     const superInfo = document.getElementById("superInfo");

#     const mgr = document.getElementById("mgr");
#     const navEl = document.getElementById("navigation");
#     const seqEl = document.getElementById("sequence");
#     const coloredEl = document.getElementById("colored-sequence");
#     const trDomain = document.getElementById("domain");
#     const domainLegend = document.getElementById("domain-legend");
#     const trRegion = document.getElementById("region");
#     const trSite = document.getElementById("site");
#     const trBinding = document.getElementById("binding");
#     const trChain = document.getElementById("chain");
#     const trDisulf = document.getElementById("disulfide-bond");
#     const trStrand = document.getElementById("beta-strand");
#     const trScop3P = document.getElementById("scop3p");
#     const variantsEl = document.getElementById("variants");

#     const structSelect = document.getElementById("structureSelect");
#     const toggleScop = document.getElementById("toggleScop3P");

#     const toggleSuper = document.getElementById("toggleSuperpose");
#     const segSel = document.getElementById("segmentSelect");
#     const cluSel = document.getElementById("clusterSelect");
#     const chkComplete = document.getElementById("toggleCompleteCluster");
#     const chkLigand = document.getElementById("toggleLigandView");
#     const ligandColor = document.getElementById("ligandColor");
#     const toggleScopSuper = document.getElementById("toggleScop3P_super");
#     const superHint = document.getElementById("superHint");

#     const mainDiv  = document.getElementById("pdbeMolstarMain");
#     const molDiv   = document.getElementById("molstarSuper");
#     const btnHideSuper = document.getElementById("btnHideSuper");
#     const btnFitMain = document.getElementById("btnFitMain");
#     const btnFitSuper = document.getElementById("btnFitSuper");

#     const PDBeMolstarPlugin = window.PDBeMolstarPlugin;
#     const molstarNS = window.molstar;

#     /* ===== 1D setup ===== */
#     await Promise.all([
#       customElements.whenDefined("nightingale-sequence"),
#       customElements.whenDefined("nightingale-navigation"),
#       customElements.whenDefined("nightingale-manager"),
#       customElements.whenDefined("nightingale-track"),
#       customElements.whenDefined("nightingale-colored-sequence"),
#       customElements.whenDefined("nightingale-linegraph-track")
#     ]);

#     const seq = featuresData.sequence;
#     const len = seq.length;
#     const setLen = (el) => { if (!el) return; el.length = len; el.setAttribute("length", String(len)); el.displayStart = 1; el.displayEnd = len; el.setAttribute("display-start","1"); el.setAttribute("display-end", String(len)); };
#     [navEl, seqEl, coloredEl, trDomain, trRegion, trSite, trBinding, trChain, trDisulf, trStrand, trScop3P, variantsEl].forEach(setLen);
#     seqEl.data = seq; coloredEl.data = seq;

#     const feats = (featuresData.features || []).map(ft => ({ ...ft, start: ft.start ?? ft.begin }));
#     trDomain.data = feats.filter(f => f.type === "DOMAIN").map(f => ({ ...f, label: f.description || f.type || "domain" }));
#     trRegion.data = feats.filter(f => f.type === "REGION");
#     trSite.data   = feats.filter(f => f.type === "SITE");
#     trBinding.data= feats.filter(f => f.type === "BINDING");
#     trChain.data  = feats.filter(f => f.type === "CHAIN");
#     trDisulf.data = feats.filter(f => f.type === "DISULFID");
#     trStrand.data = feats.filter(f => f.type === "STRAND");

#     try {
#       const items = trDomain.data.slice(0, 12).map(d => `${d.label || d.description || "domain"} (${d.start}-${d.end})`);
#       domainLegend.textContent = items.length ? "Domains: " + items.join("·") + (trDomain.data.length > 12 ? " …" : "") : "";
#     } catch {}

#     // Variants sparkline
#     const counts = new Map();
#     for (const v of (variationData.features || [])) {
#       const pos = Number(v.begin);
#       if (!Number.isFinite(pos)) continue;
#       counts.set(pos, (counts.get(pos) || 0) + 1);
#     }
#     const values = Array.from(counts, ([position, value]) => ({ position, value })).sort((a,b)=>a.position-b.position);
#     const max = values.length ? Math.max(...values.map(d=>d.value)) : 0;
#     variantsEl.data = [{ color: "grey", values, range: [0, Math.max(1, max)] }];

#     trScop3P.data = (scop3pCache[accession]?.modifications || [])
#       .filter(m => String(m.name).toLowerCase()==="phosphorylation")
#       .filter(m => Number.isFinite(Number(m.position)))
#       .map(m => ({ start: +m.position, end: +m.position, type: "SCOP3P_PHOSPHO", description: `Phospho ${(m.residue||'?').slice(0,1)}@${m.position} (${m.source||'Scop3P'})` }));

#     /* ===== Left viewer (PDBe wrapper) ===== */
#     function populateStructureSelect(arr) {
#       structSelect.innerHTML = "";
#       const afFirst = arr.slice().sort((a,b) => (a.source === "AFDB" ? -1 : (b.source === "AFDB" ? 1 : 0)));
#       for (const s of afFirst) {
#         const opt = document.createElement("option");
#         opt.value = (s.source === "PDB" && s.chain) ? `${s.id}|${s.chain}` : s.id;
#         opt.textContent = s.label || (s.source === "PDB" ? `${s.id}${s.chain? " — chain "+s.chain : ""}` : s.id);
#         opt.dataset.source = s.source || "";
#         structSelect.appendChild(opt);
#       }
#       const afIdx = Array.from(structSelect.options).findIndex(o => o.value.startsWith(`AF-${accession}-F1`));
#       structSelect.selectedIndex = afIdx >= 0 ? afIdx : 0;
#     }
#     populateStructureSelect(structures);

#     let viewerMain = null;
#     async function waitSize(el) { return new Promise(r => {
#       let ok = 0;
#       const f = () => { const b = el.getBoundingClientRect(); if (b.width>0 && b.height>0) { ok++; if (ok>=2) return r(); } else ok=0; requestAnimationFrame(f); };
#       const ro = new ResizeObserver(() => { const b = el.getBoundingClientRect(); if (b.width>0 && b.height>0) { ok++; if (ok>=2) { ro.disconnect(); r(); } }});
#       ro.observe(el); f();
#     }); }

#     async function choosePdbCustomData(pdbIdUpper) {
#       const id = pdbIdUpper.toLowerCase();
#       const bcif = `https://www.ebi.ac.uk/pdbe/entry-files/download/${id}.bcif`;
#       const cif  = `https://www.ebi.ac.uk/pdbe/entry-files/download/${id}.cif`;
#       try { const r = await fetch(bcif, { method: "HEAD" }); if (r && r.ok) return { url: bcif, format: "bcif", binary: true }; } catch {}
#       return { url: cif, format: "cif", binary: false };
#     }

#     async function renderLeft() {
#       await waitSize(mainDiv);
#       if (viewerMain?.plugin?.dispose) { try { viewerMain.plugin.dispose(); } catch {} }
#       const inst = new PDBeMolstarPlugin();
#       const opt = structSelect.options[structSelect.selectedIndex];
#       const val = opt ? opt.value : "";
#       const src = opt ? (opt.dataset.source || "") : "";
#       if (src === "AFDB" || val.startsWith(`AF-${accession}-F1`)) {
#         inst.render(mainDiv, { customData: { url: `https://alphafold.ebi.ac.uk/files/AF-${accession}-F1-model_v4.cif`, format: "cif", binary: false }, hideControls: true, bgColor: { r:255,g:255,b:255 }, alphafoldView: false });
#       } else {
#         const pdbId = val.split("|")[0] || "";
#         if (!pdbId) return;
#         const customData = await choosePdbCustomData(pdbId);
#         inst.render(mainDiv, { customData, hideControls: true, bgColor: { r:255,g:255,b:255 } });
#       }
#       viewerMain = inst;
#       const buildScop = () => (scop3pCache[accession]?.modifications||[])
#         .filter(m=>String(m.name).toLowerCase()==="phosphorylation")
#         .filter(m=>Number.isFinite(+m.position))
#         .map(m=>({ uniprot_accession: accession, start_uniprot_residue_number:+m.position, end_uniprot_residue_number:+m.position, color:{r:255,g:0,b:0}, representation:"ball-and-stick", sideChain:true }));
#       const applyLeft = () => {
#         if (!viewerMain?.visual?.select) return;
#         const data = toggleScop.checked ? buildScop() : [];
#         viewerMain.visual.select({ data, nonSelectedColor: { r:176,g:176,b:176 } });
#       };
#       if (viewerMain?.events?.loadComplete?.subscribe) {
#         const sub = viewerMain.events.loadComplete.subscribe(() => { applyLeft(); setTimeout(applyLeft, 250); try { sub.unsubscribe(); } catch {} });
#       } else setTimeout(applyLeft, 400);
#     }

#     structSelect.addEventListener("change", renderLeft);
#     toggleScop.addEventListener("change", () => { try {
#       if (!viewerMain?.visual?.select) return;
#       const data = (toggleScop.checked ? (scop3pCache[accession]?.modifications||[])
#         .filter(m=>String(m.name).toLowerCase()==="phosphorylation")
#         .filter(m=>Number.isFinite(+m.position))
#         .map(m=>({ uniprot_accession: accession, start_uniprot_residue_number:+m.position, end_uniprot_residue_number:+m.position, color:{r:255,g:0,b:0}, representation:"ball-and-stick", sideChain:true })) : []);
#       viewerMain.visual.select({ data, nonSelectedColor:{r:176,g:176,b:176} });
#     } catch {} });

#     await renderLeft();

#     /* ===== Right viewer (Mol* core) ===== */
#     let molViewer = null;
#     async function ensureMolViewer() {
#       await waitSize(molDiv);
#       if (molViewer) return molViewer;
#       molViewer = new molstarNS.Viewer(molDiv, {
#         layoutIsExpanded: true,
#         layoutShowControls: false,
#         layoutShowSequence: false,
#         layoutShowLog: false,
#         layoutShowLeftPanel: false,
#         viewportShowExpand: false,
#         backgroundColor: { r: 255, g: 255, b: 255 }
#       });
#       return molViewer;
#     }

#     // PDBe Graph API: superposition segments
#     async function fetchSuperpositionGraph(uniprot) {
#       try {
#         const url = "https://www.ebi.ac.uk/pdbe/graph-api/uniprot/superposition/" + uniprot;
#         const res = await fetch(url, { cache: "no-store" });
#         if (!res.ok) return [];
#         const js = await res.json();
#         const arr = js && js[uniprot] ? js[uniprot] : [];
#         if (!Array.isArray(arr)) return [];
#         const out = [];
#         for (let i=0; i<arr.length; i++) {
#           const seg = arr[i];
#           const segId = i + 1;
#           const clusters = [];
#           const raws = Array.isArray(seg.clusters) ? seg.clusters : [];
#           for (let k=0; k<raws.length; k++) {
#             const list = raws[k] || [];
#             const repr = list.find(x => x?.is_representative) || list[0] || null;
#             clusters.push({ id: k + 1, size: list.length, list, repr });
#           }
#           out.push({ id: segId, start: +seg.segment_start, end: +seg.segment_end, clusters });
#         }
#         return out;
#       } catch { return []; }
#     }

#     // PDBe mappings (SIFTS): UniProt → PDB residue/chain map
#     async function fetchUniprotPdbMappings(uniprot) {
#       // Returns: mapping[pdbIdUpper].chains[auth_asym_id] = [ { unp_start, unp_end, start:{residue_number}, end:{residue_number} }, ... ]
#       const url = "https://www.ebi.ac.uk/pdbe/api/mappings/uniprot/" + uniprot;
#       try {
#         const res = await fetch(url);
#         if (!res.ok) return {};
#         const js = await res.json();
#         return js?.[uniprot?.toUpperCase?.() || uniprot] || {};
#       } catch { return {}; }
#     }

#     function populateSegmentsUI(segments) {
#       segSel.innerHTML = "";
#       for (const s of segments) {
#         const opt = document.createElement("option");
#         opt.value = String(s.id);
#         opt.textContent = `Segment ${s.id} (${s.start}-${s.end})`;
#         segSel.appendChild(opt);
#       }
#       segSel.disabled = segments.length === 0;
#     }
#     function populateClustersUI(segments, segId) {
#       const s = segments.find(x => x.id == segId);
#       const cls = (s && s.clusters) ? s.clusters : [];
#       cluSel.innerHTML = "";
#       for (const c of cls) {
#         const opt = document.createElement("option");
#         const r = c.repr;
#         const reprTxt = (r && r.pdb_id) ? ` repr ${r.pdb_id}:${(r.auth_asym_id || r.struct_asym_id || "")}` : "";
#         opt.value = String(c.id);
#         opt.textContent = `Cluster ${c.id} (n=${c.size})${reprTxt}`;
#         opt.dataset.repr = JSON.stringify(r || null);
#         cluSel.appendChild(opt);
#       }
#       cluSel.disabled = cls.length === 0;
#     }

#     // Kabsch alignment helper
#     function kabschFit(P, Q) {
#       const centroid = A => {
#         const c = [0,0,0]; for (const v of A) { c[0]+=v[0]; c[1]+=v[1]; c[2]+=v[2]; }
#         c[0]/=A.length; c[1]/=A.length; c[2]/=A.length; return c;
#       };
#       const sub = (A,c) => A.map(v => [v[0]-c[0], v[1]-c[1], v[2]-c[2]]);
#       const Ct = centroid(P), Cu = centroid(Q);
#       const X = sub(P, Ct), Y = sub(Q, Cu);
#       const H = [[0,0,0],[0,0,0],[0,0,0]];
#       for (let i=0;i<X.length;i++) for (let r=0;r<3;r++) for (let c=0;c<3;c++) H[r][c] += X[i][r]*Y[i][c];

#       function svd3x3(M){
#         const mtm = (A) => {
#           const T=[[A[0][0],A[1][0],A[2][0]],[A[0][1],A[1][1],A[2][1]],[A[0][2],A[1][2],A[2][2]]];
#           const R=[[0,0,0],[0,0,0],[0,0,0]];
#           for(let i=0;i<3;i++)for(let j=0;j<3;j++)for(let k=0;k<3;k++)R[i][j]+=T[i][k]*A[k][j];
#           return R;
#         };
#         function eigSym(A, iters=32){
#           const V=[[1,0,0],[0,1,0],[0,0,1]];
#           const vals=[0,0,0];
#           for (let k=0;k<3;k++){
#             let v=[Math.random(),Math.random(),Math.random()];
#             for (let t=0;t<iters;t++){
#               const w=[A[0][0]*v[0]+A[0][1]*v[1]+A[0][2]*v[2],
#                        A[1][0]*v[0]+A[1][1]*v[1]+A[1][2]*v[2],
#                        A[2][0]*v[0]+A[2][1]*v[1]+A[2][2]*v[2]];
#               const n=Math.hypot(w[0],w[1],w[2])||1; v=[w[0]/n,w[1]/n,w[2]/n];
#             }
#             const lambda=v[0]*(A[0][0]*v[0]+A[0][1]*v[1]+A[0][2]*v[2]) + v[1]*(A[1][0]*v[0]+A[1][1]*v[1]+A[1][2]*v[2]) + v[2]*(A[2][0]*v[0]+A[2][1]*v[1]+A[2][2]*v[2]);
#             vals[k]=lambda;
#             V[0][k]=v[0]; V[1][k]=v[1]; V[2][k]=v[2];
#           }
#           return {vals, V};
#         }
#         const MtM = mtm(M);
#         const { vals, V } = eigSym(MtM);
#         const idx=[0,1,2].sort((a,b)=>vals[b]-vals[a]);
#         const S=[Math.sqrt(Math.max(vals[idx[0]],0)), Math.sqrt(Math.max(vals[idx[1]],0)), Math.sqrt(Math.max(vals[idx[2]],0))];
#         const Vsort=[[V[0][idx[0]],V[0][idx[1]],V[0][idx[2]]],
#                      [V[1][idx[0]],V[1][idx[1]],V[1][idx[2]]],
#                      [V[2][idx[0]],V[2][idx[1]],V[2][idx[2]]]];
#         const Sinv = [ S[0]?1/S[0]:0, S[1]?1/S[1]:0, S[2]?1/S[2]:0 ];
#         const HV=[[0,0,0],[0,0,0],[0,0,0]];
#         for(let i=0;i<3;i++)for(let j=0;j<3;j++)for(let k=0;k<3;k++)HV[i][j]+=M[i][k]*Vsort[k][j];
#         const U=[[0,0,0],[0,0,0],[0,0,0]];
#         for(let i=0;i<3;i++)for(let j=0;j<3;j++)U[i][j]=HV[i][j]*Sinv[j];
#         return {U, S, V: Vsort};
#       }
#       const {U,S,V} = svd3x3(H);
#       const Ut=[[U[0][0],U[1][0],U[2][0]],[U[0][1],U[1][1],U[2][1]],[U[0][2],U[1][2],U[2][2]]];
#       let R=[[0,0,0],[0,0,0],[0,0,0]];
#       for(let i=0;i<3;i++)for(let j=0;j<3;j++)for(let k=0;k<3;k++)R[i][j]+=V[i][k]*Ut[k][j];
#       const det = R[0][0]*(R[1][1]*R[2][2]-R[1][2]*R[2][1]) - R[0][1]*(R[1][0]*R[2][2]-R[1][2]*R[2][0]) + R[0][2]*(R[1][0]*R[2][1]-R[1][1]*R[2][0]);
#       if (det < 0) {
#         for(let i=0;i<3;i++) V[i][2] *= -1;
#         R=[[0,0,0],[0,0,0],[0,0,0]];
#         for(let i=0;i<3;i++)for(let j=0;j<3;j++)for(let k=0;k<3;k++)R[i][j]+=V[i][k]*Ut[k][j];
#       }
#       const Ct=[...X.reduce((a,b)=>[a[0]+b[0],a[1]+b[1],a[2]+b[2]],[0,0,0])].map(v=>v/X.length);
#       const Cu=[...Y.reduce((a,b)=>[a[0]+b[0],a[1]+b[1],a[2]+b[2]],[0,0,0])].map(v=>v/Y.length);
#       const t=[ Cu[0]- (R[0][0]*Ct[0]+R[0][1]*Ct[1]+R[0][2]*Ct[2]),
#                 Cu[1]- (R[1][0]*Ct[0]+R[1][1]*Ct[1]+R[1][2]*Ct[2]),
#                 Cu[2]- (R[2][0]*Ct[0]+R[2][1]*Ct[1]+R[2][2]*Ct[2]) ];
#       return { R, t };
#     }

#     async function ensureMol() {
#       try { return await ensureMolViewer(); } catch (e) { console.error(e); throw e; }
#     }

#     async function loadAndSuperpose({ segment, clusters }) {
#       const segObj = segmentsMeta.find(s => s.id == segment);
#       if (!segObj) throw new Error("No segment object");
#       const startU = segObj.start, endU = segObj.end;

#       const mapping = await fetchUniprotPdbMappings(accession);

#       const reps = [];
#       const segClusters = segmentsMeta.find(s => s.id == segment)?.clusters || [];
#       const selIds = new Set((clusters||[]).map(Number));
#       const chosen = segClusters.filter(c => selIds.size ? selIds.has(c.id) : true);
#       for (const c of chosen) {
#         if (!c.repr) continue;
#         const rep = c.repr; // { pdb_id, auth_asym_id / struct_asym_id }
#         const pdb = (rep.pdb_id || "").toLowerCase();
#         const chain = rep.auth_asym_id || rep.struct_asym_id;
#         if (!pdb || !chain) continue;
#         const mapForPdb = mapping[pdb?.toUpperCase?.() || pdb];
#         const byChain = mapForPdb?.chains?.[chain];
#         const segs = (byChain || []).filter(s => !(s.unp_end < startU || s.unp_start > endU));
#         if (!segs.length) continue;
#         reps.push({ pdb, chain, sifts: segs });
#       }

#       const viewer = await ensureMol();
#       await viewer.clear();

#       if (!reps.length) throw new Error("No representative with SIFTS mapping found.");
#       const ref = reps[0];

#       async function loadPdb(pdbId) {
#         const urlBCIF = `https://www.ebi.ac.uk/pdbe/entry-files/download/${pdbId}.bcif`;
#         const urlCIF  = `https://www.ebi.ac.uk/pdbe/entry-files/download/${pdbId}.cif`;
#         try { const h = await fetch(urlBCIF, { method:"HEAD" }); if (h.ok) return viewer.loadStructureFromUrl(urlBCIF, 'bcif', true); } catch {}
#         return viewer.loadStructureFromUrl(urlCIF, 'cif', false);
#       }

#       const modelRef = await loadPdb(ref.pdb);

#       function getCaCoords(model, pdbId, chainId, siftsSegs) {
#         const wanted = new Set();
#         for (const s of siftsSegs) {
#           const u0 = Math.max(startU, s.unp_start), u1 = Math.min(endU, s.unp_end);
#           const p0 = s.start?.residue_number, p1 = s.end?.residue_number;
#           if (Number.isFinite(p0) && Number.isFinite(p1) && u1>=u0) {
#             const spanU = s.unp_end - s.unp_start;
#             const spanP = p1 - p0;
#             for (let u = u0; u <= u1; u++) {
#               const alpha = spanU ? (u - s.unp_start) / spanU : 0;
#               const p = Math.round(p0 + alpha * spanP);
#               wanted.add(p);
#             }
#           }
#         }
#         const plugin = viewer.plugin;
#         const structure = plugin.managers.structure.hierarchy.current.structures[0]?.cell?.obj?.data;
#         if (!structure) return [];
#         const coords = [];
#         for (const { units } of structure.structures || []) {
#           for (const u of units) {
#             const props = u.model.atomicHierarchy;
#             if (!props) continue;
#             const chainIndices = props.chains.asym_id;
#             const authSeq = props.residues.auth_seq_id;
#             const atomType = props.atoms.label_atom_id;
#             const atomResidue = props.atoms.residueIndex;
#             const atomChain = props.atoms.chainIndex;
#             const x = u.conformation.x, y = u.conformation.y, z = u.conformation.z;
#             for (let aI = 0; aI < atomResidue.length; aI++) {
#               if (atomType.value(aI) !== 'CA') continue;
#               const cIdx = atomChain.value(aI);
#               if (chainIndices.value(cIdx) !== chainId) continue;
#               const rIdx = atomResidue.value(aI);
#               const auth = authSeq.value(rIdx);
#               if (!wanted.has(auth)) continue;
#               coords.push([ x(aI), y(aI), z(aI) ]);
#             }
#           }
#         }
#         return coords;
#       }

#       const refCoords = getCaCoords(modelRef, ref.pdb, ref.chain, ref.sifts);

#       await viewer.setStructureProperties({ ignoreLight: false });
#       await viewer.addRepresentation(modelRef, { type: 'cartoon', color: 'uniform', colorParams: { value: 0xB0B0B0 } });

#       async function applyTransformToModel(model, R, t) {
#         const trans = {
#           matrix: [
#             R[0][0], R[0][1], R[0][2], 0,
#             R[1][0], R[1][1], R[1][2], 0,
#             R[2][0], R[2][1], R[2][2], 0,
#             t[0],    t[1],    t[2],    1
#           ]
#         };
#         await viewer.transform(model, trans);
#       }

#       for (let i=1; i<reps.length; i++) {
#         const it = reps[i];
#         const model = await loadPdb(it.pdb);
#         const movCoords = getCaCoords(model, it.pdb, it.chain, it.sifts);
#         const N = Math.min(refCoords.length, movCoords.length);
#         if (N >= 4) {
#           const P = movCoords.slice(0, N);
#           const Q = refCoords.slice(0, N);
#           const { R, t } = kabschFit(P, Q);
#           await applyTransformToModel(model, R, t);
#         }
#         await viewer.addRepresentation(model, { type: 'cartoon', color: 'uniform', colorParams: { value: 0xB0B0B0 } });
#       }

#       function scop3pPositionsInRange(start, end) {
#         const mods = (scop3pCache[accession]?.modifications || [])
#           .filter(m => String(m.name).toLowerCase() === 'phosphorylation')
#           .map(m => +m.position)
#           .filter(n => Number.isFinite(n) && n >= start && n <= end);
#         return new Set(mods);
#       }
#       const phospho = scop3pPositionsInRange(startU, endU);

#       async function addSticksForUniprotPositionsOnModel(model, pdbId, chainId, siftsSegs) {
#         const wantedAuthSeq = new Set();
#         for (const s of siftsSegs) {
#           const u0 = Math.max(startU, s.unp_start), u1 = Math.min(endU, s.unp_end);
#           const p0 = s.start?.residue_number, p1 = s.end?.residue_number;
#           if (!Number.isFinite(p0) || !Number.isFinite(p1)) continue;
#           const spanU = s.unp_end - s.unp_start;
#           const spanP = p1 - p0;
#           for (const u of phospho) {
#             if (u < u0 || u > u1) continue;
#             const alpha = spanU ? (u - s.unp_start) / spanU : 0;
#             const p = Math.round(p0 + alpha * spanP);
#             wantedAuthSeq.add(p);
#           }
#         }
#         if (!wantedAuthSeq.size) return;

#         const plugin = viewer.plugin;
#         const structure = plugin.managers.structure.hierarchy.current.structures.find(s => {
#           const label = s.cell?.obj?.data?.label || '';
#           return label.toLowerCase().includes(pdbId);
#         })?.cell?.obj?.data;
#         if (!structure) return;

#         const loci = [];
#         for (const { units } of structure.structures || []) {
#           for (const u of units) {
#             const props = u.model.atomicHierarchy;
#             if (!props) continue;
#             const chainIndices = props.chains.asym_id;
#             if (!chainIndices) continue;
#             const authSeq = props.residues.auth_seq_id;
#             const atomResidue = props.atoms.residueIndex;
#             const atomChain = props.atoms.chainIndex;
#             const atomType = props.atoms.label_atom_id;
#             const elements = [];
#             for (let aI = 0; aI < atomResidue.length; aI++) {
#               const cIdx = atomChain.value(aI);
#               if (chainIndices.value(cIdx) !== chainId) continue;
#               const rIdx = atomResidue.value(aI);
#               const auth = authSeq.value(rIdx);
#               if (!wantedAuthSeq.has(auth)) continue;
#               if (atomType.value(aI) === 'CA') continue;
#               elements.push(aI);
#             }
#             if (elements.length) loci.push({ unit: u, indices: elements });
#           }
#         }
#         if (!loci.length) return;
#         await viewer.addRepresentation(model, { type: 'ball-and-stick', color: 'uniform', colorParams: { value: 0xFF0000 } }, loci);
#       }

#       await addSticksForUniprotPositionsOnModel(modelRef, ref.pdb, ref.chain, ref.sifts);
#       await viewer.autoView();
#       return { count: reps.length, seg: segObj };
#     }

#     /* ==== UI wiring for right panel ==== */
#     let segmentsMeta = [];
#     molDiv.style.display = "none";

#     const segments = await fetchSuperpositionGraph(accession);
#     segmentsMeta = segments;

#     if (!segments.length) {
#       toggleSuper.disabled = true;
#       [segSel, cluSel, chkComplete, chkLigand, ligandColor, toggleScopSuper].forEach(el => el.disabled = true);
#       warnEl.textContent = "Superposition not available (no PDBe-KB segments).";
#     } else {
#       toggleSuper.disabled = false;
#       warnEl.textContent = "";
#       populateSegmentsUI(segments);
#       segSel.value = String(segments[0].id);
#       populateClustersUI(segments, segSel.value);
#     }

#     async function renderRightFromUI() {
#       if (!toggleSuper.checked) return;
#       molDiv.style.display = "block";
#       const segId = parseInt(segSel.value, 10);
#       const clusters = Array.from(cluSel.selectedOptions).map(o => parseInt(o.value, 10)).filter(Number.isFinite);
#       superHint.textContent = clusters.length ? `Segment ${segId}, clusters [${clusters.join(", ")}]`
#                                               : `Segment ${segId} (all)`;
#       superInfo.textContent = "aligning…";
#       try {
#         const { count, seg } = await loadAndSuperpose({ segment: segId, clusters });
#         superInfo.textContent = `loaded ${count} rep(s) — ${seg.start}-${seg.end}`;
#       } catch (e) {
#         superInfo.textContent = "error";
#         console.error(e);
#         warnEl.textContent = "Superposition failed: " + (e?.message || String(e));
#       }
#     }

#     document.getElementById("toggleSuperpose").addEventListener("change", async () => {
#       if (toggleSuper.checked) await renderRightFromUI();
#       else { molDiv.style.display = "none"; superHint.textContent = ""; superInfo.textContent = ""; }
#     });

#     document.getElementById("btnHideSuper").addEventListener("click", () => {
#       toggleSuper.checked = false;
#       molDiv.style.display = "none";
#       superHint.textContent = "";
#       superInfo.textContent = "";
#     });

#     btnFitSuper.addEventListener("click", async () => {
#       if (molViewer) await molViewer.autoView();
#     });

#     segSel.addEventListener("change", async () => {
#       populateClustersUI(segmentsMeta, segSel.value);
#       await renderRightFromUI();
#     });
#     cluSel.addEventListener("change", async () => { await renderRightFromUI(); });

#     // ===== Simple 1D↔left overlays (unchanged) =====
#     const obs = new MutationObserver(() => { /* left viewer highlight re-apply already handled */ });
#     obs.observe(navEl, { attributes: true, attributeFilter: ["highlight"] });
#     mgr.addEventListener("click", () => { /* left overlays already handled */ });

#     btnFitMain.addEventListener("click", () => { try { viewerMain?.visual?.reset({ camera: true }); } catch {} });
#   }

#   /* 3) Call main() without await (no top-level await) */
#   main();
# </script>
# </body>
# </html>
# """

# def build_html(acc: str, out_dir: str = ".") -> Path:
#     b = build_bundle(acc)
#     features_cache   = json.dumps({acc: b["features"]}, ensure_ascii=False)
#     variation_cache  = json.dumps({acc: b["variation"]}, ensure_ascii=False)
#     structures_cache = json.dumps({acc: b["structures"]}, ensure_ascii=False)
#     scop3p_cache     = json.dumps({acc: b["scop3p"]}, ensure_ascii=False)
#     scop3p_compact   = json.dumps({acc: b["scop3p_compact"]}, ensure_ascii=False)

#     html_out = (
#         HTML_TEMPLATE
#         .replace("__ACC__", acc)
#         .replace("__FEATURES_CACHE__", features_cache)
#         .replace("__VARIATION_CACHE__", variation_cache)
#         .replace("__STRUCTURES_CACHE__", structures_cache)
#         .replace("__SCOP3P_CACHE__", scop3p_cache)
#         .replace("__SCOP3P_COMPACT__", scop3p_compact)
#     )
#     out_path = Path(out_dir) / f"nightingale_molstar_super_{acc}.html"
#     out_path.write_text(html_out, encoding="utf-8")
#     return out_path


In [2]:
# p = build_html("P07949"); p

In [ ]:
# Biophysical properites as track: first predicted for the UniProt ID and then added as a track in 1D and 3D 

In [11]:
# Cell 1 additions (top)
import numpy as np, tempfile
from b2bTools import SingleSeq, constants
import requests


In [44]:
# Cell 1: Fetch data server-side and normalize it (no browser CORS)
import json, re, requests
from typing import Dict, Any, List, Tuple

SESS = requests.Session()
SESS.headers.update({"User-Agent": "nightingale-pdbe-builder/1.1"})

def _get(url: str, timeout=30):
    r = SESS.get(url, timeout=timeout)
    r.raise_for_status()
    return r

def fetch_json(url: str, timeout: int = 30) -> Dict[str, Any]:
    return _get(url, timeout).json()

def fetch_features(acc: str) -> Dict[str, Any]:
    return fetch_json(f"https://www.ebi.ac.uk/proteins/api/features/{acc}")

def fetch_variation(acc: str) -> Dict[str, Any]:
    try:
        return fetch_json(f"https://www.ebi.ac.uk/proteins/api/variation/{acc}")
    except Exception:
        return {"features": []}

def fetch_uniprot(acc: str) -> Dict[str, Any]:
    return fetch_json(f"https://rest.uniprot.org/uniprotkb/{acc}.json")

def fetch_scop3p(acc: str) -> Dict[str, Any]:
    return fetch_json(f"https://iomics.ugent.be/scop3p/api/modifications?accession={acc}")

def _iter_pdb_xrefs(u: Dict[str, Any]):
    xs = u.get("uniProtKBCrossReferences") or []
    if xs:
        for x in xs:
            if (x.get("database") or x.get("type")) == "PDB":
                yield {"id": x.get("id"), "properties": x.get("properties") or []}
    legacy = u.get("dbReferences") or []
    for x in legacy:
        if x.get("type") == "PDB":
            yield {"id": x.get("id"), "properties": x.get("properties") or []}

def parse_pdb_structures(uniprot_json: Dict[str, Any]) -> List[Dict[str, Any]]:
    """
    Return unique PDB items: either per-chain (when UniProt gives chain mapping text) or
    a plain PDB entry when no chain text is available.
    """
    seen = set()
    out: List[Dict[str, Any]] = []
    for xr in _iter_pdb_xrefs(uniprot_json):
        pdb_id = xr.get("id")
        if not pdb_id:
            continue
        props = { (p.get("type") or p.get("key")): p.get("value") for p in xr.get("properties", []) }
        chains_str = (props.get("chains") or props.get("Chains") or "").strip()
        added_chain = False
        if chains_str:
            # Parse "A=25-300, 310-350; B=..." style
            parts = re.split(r"[;,]\s*", chains_str)
            for part in filter(None, parts):
                m = re.match(r"^([A-Za-z0-9]+)\s*=\s*([0-9,\-\s]+)$", part)
                if not m:
                    continue
                chain = m.group(1).strip()
                key = (pdb_id.upper(), chain)
                if key in seen:
                    continue
                seen.add(key)
                out.append({
                    "id": pdb_id.upper(),
                    "source": "PDB",
                    "chain": chain,
                    "label": f"{pdb_id.upper()} — chain {chain}",
                })
                added_chain = True
        if not added_chain:
            # Keep a plain entry (no chain mapping disclosed by UniProt)
            key = (pdb_id.upper(), None)
            if key not in seen:
                seen.add(key)
                out.append({
                    "id": pdb_id.upper(),
                    "source": "PDB",
                    "chain": None,
                    "label": pdb_id.upper(),
                })
    # Sort AF/PDB later; here just by id/chain
    out.sort(key=lambda d: (d["id"], "" if d["chain"] is None else str(d["chain"])))
    return out

def merge_positions_to_ranges(positions: List[int]) -> List[Tuple[int, int]]:
    if not positions:
        return []
    s = sorted(set(int(p) for p in positions))
    ranges: List[Tuple[int,int]] = []
    start = prev = s[0]
    for p in s[1:]:
        if p == prev + 1:
            prev = p
        else:
            ranges.append((start, prev))
            start = prev = p
    ranges.append((start, prev))
    return ranges


# === Biophys helpers ===
def fetch_sequence_aminoacids(accession: str):
    """Fetch FASTA from UniProt (sequence-only)."""
    url = f"https://rest.uniprot.org/uniprotkb/{accession}.fasta"
    r = requests.get(url, timeout=30)
    if r.status_code != 200:
        url_legacy = f"https://www.uniprot.org/uniprotkb/{accession}.fasta?accession={accession}"
        r = requests.get(url_legacy, timeout=30)
        r.raise_for_status()
    raw = r.content.decode("utf-8")
    lines = [ln.strip() for ln in raw.splitlines() if ln.strip()]
    header = lines[0] if lines and lines[0].startswith(">") else f">{accession}"
    seq = "".join(ln for ln in lines if not ln.startswith(">"))
    return header, seq

def predict_biophysical_features(accession: str, sequence: str):
    """Run DynaMine/DisoMine/EfoldMine on a single sequence via b2bTools."""
    with tempfile.NamedTemporaryFile(prefix="seq_", suffix=".fasta", mode="w") as fp:
        fp.write(f">{accession}\n{sequence}\n")
        fp.flush()
        pred = SingleSeq(fp.name).predict(
            tools=[constants.TOOL_DYNAMINE, constants.TOOL_DISOMINE, constants.TOOL_EFOLDMINE]
        ).get_all_predictions()
    data = pred["proteins"][accession]
    # Ensure list[float] with same len as sequence
    seq_pred = data["seq"]
    bb = [float(x) for x in data["backbone"]]
    diso = [float(x) for x in data["disoMine"]]
    ef = [float(x) for x in data["earlyFolding"]]
    assert len(seq_pred) == len(sequence) == len(bb) == len(diso) == len(ef)
    return {"backbone": bb, "disorder": diso, "efold": ef}

def _bucket_backbone(v: float) -> int:
    # 1=membrane/super-rigid (>1.0), 2=rigid (>0.8), 3=context (>0.69), 4=flexible (else)
    return 1 if v > 1.0 else 2 if v > 0.8 else 3 if v > 0.69 else 4


def build_bundle(acc: str) -> Dict[str, Any]:
    features = fetch_features(acc)
    variation = fetch_variation(acc)
    uniprot = fetch_uniprot(acc)
    scop3p = fetch_scop3p(acc)

    # AlphaFold first
    structures = [{"id": f"AF-{acc}-F1", "source": "AFDB", "chain": None, "label": f"AlphaFold (AF-{acc}-F1)"}]
    structures.extend(parse_pdb_structures(uniprot))

    phospho = [m for m in (scop3p.get("modifications") or []) if str(m.get("name","")).lower() == "phosphorylation"]
    phospho_sites = [
        {"pos": int(m["position"]), "residue": (m.get("residue") or "?")[:1], "source": m.get("source") or "Scop3P"}
        for m in phospho if "position" in m
    ]
    highlight_ranges = merge_positions_to_ranges([p["pos"] for p in phospho_sites])
    highlight_str = ",".join(f"{s}:{e}" for (s, e) in highlight_ranges)

    # === NEW: biophysical predictions ===
    try:
        _, seq = fetch_sequence_aminoacids(acc)
        bio = predict_biophysical_features(acc, seq)
        # print ('inside biophys',bio.keys())
        # print (seq)
        # Precompute buckets for the main color toggle (keeps client light)
        bbuckets = [_bucket_backbone(v) for v in bio["backbone"]]
        biophys_compact = {
            "length": len(seq),
            "backbone": bio["backbone"],
            "disorder": bio["disorder"],
            "efold": bio["efold"],
            "backbone_buckets": bbuckets,   # 1..4
            "ranges": {
                "backbone": [float(min(bio["backbone"])), float(max(bio["backbone"]))],
                "disorder": [float(min(bio["disorder"])), float(max(bio["disorder"]))],
                "efold":    [float(min(bio["efold"])), float(max(bio["efold"]))],
            },
        }
    except Exception as e:
        # Fail-soft: expose empty; UI hides gracefully
        biophys_compact = {"length": 0, "backbone": [], "disorder": [], "efold": [], "backbone_buckets": [], "ranges": {}}

    return {
        "accession": acc,
        "features": features,
        "variation": variation,
        "structures": structures,
        "scop3p": scop3p,
        "scop3p_compact": {"phospho_sites": phospho_sites, "highlight": highlight_str},
        "biophys_compact": biophys_compact,  # NEW
    }



In [61]:
from pathlib import Path
import json

HTML_TEMPLATE = """<!doctype html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width, initial-scale=1"/>
<title>Nightingale + PDBe Mol* — __ACC__</title>

<style>
  :root { --top-height: 34vh; --gap: 16px; --panel-height: 560px; --superpanel-height: 420px; }
  html, body { height: auto !important; overflow: auto; }
  body { margin: 0; font-family: system-ui, -apple-system, Segoe UI, Roboto, sans-serif; }
  .grid { display: grid; grid-template-rows: var(--top-height) auto 1fr; grid-template-columns: 1fr; gap: var(--gap); padding: var(--gap); align-items: stretch; }
  .top-1d { background: #fff; border: 1px solid #e5e7eb; border-radius: 8px; overflow: auto; padding: 8px; min-height: 0; }
  .center-toggle { display:flex; justify-content:center; align-items:center; padding: 0 0 4px; }
  .bottom-3d { display:grid; grid-template-columns: 1fr 1fr; gap: var(--gap); min-height:0; }
  .panel { position:relative; display:flex; flex-direction:column; gap:10px; min-height:0; background:#fff; border:1px solid #e5e7eb; border-radius:8px; padding:10px; }
  .controls { position:relative; z-index:2; background:#fff; display:flex; gap:12px; align-items:center; flex-wrap:wrap; }
  .titlebar { display:flex; align-items:center; justify-content:space-between; }
  .titlebar h3 { margin:0; font-size:1rem; color:#111827; }
  .btn { padding:6px 10px; border:1px solid #e5e7eb; border-radius:8px; background:#f9fafb; cursor:pointer; }
  .btn:hover { background:#f3f4f6; }
  label { font-size:0.9rem; color:#374151; }
  .hint { color:#374151; font-size:0.9rem; padding:0 16px 8px; }
  .warn { color:#b45309; font-size:0.85rem; padding:4px 16px 8px; min-height:1.2em; }
  table { border-collapse:collapse; width:100%; }
  td { padding:5px; vertical-align:top; }
  td:first-child { background-color: lightcyan; font:0.8em sans-serif; white-space:nowrap; }
  td:nth-child(2) { background-color: aliceblue; }
  tr:nth-child(-n + 3) > td { background-color: transparent; }
  .viewer { position:relative; width:100%; border:1px solid #e5e7eb; border-radius:8px; overflow:hidden; min-height:0; }
  #pdbeMolstarMain  { height: var(--panel-height); min-height: var(--panel-height); }
  #pdbeMolstarSuper { height: var(--superpanel-height); min-height: var(--superpanel-height); display:none; }
  .viewer .msp-plugin, .viewer .msp-viewport { position:absolute; inset:0; }
  .legend { font-size:12px; color:#374151; margin:6px 0; }
  select { padding:4px 6px; }
  input[type="color"] { width:28px; height:28px; padding:0; border:1px solid #e5e7eb; border-radius:6px; }
  .flex-spacer { flex:1; }
</style>

<script type="importmap">
{ "imports": { "@nightingale-elements/": "https://cdn.jsdelivr.net/npm/@nightingale-elements/" } }
</script>

<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/pdbe-molstar@3.8.0/build/pdbe-molstar.css">
<script src="https://cdn.jsdelivr.net/npm/pdbe-molstar@3.8.0/build/pdbe-molstar-plugin.js"></script>

<script id="features-cache"   type="application/json">__FEATURES_CACHE__</script>
<script id="variation-cache"  type="application/json">__VARIATION_CACHE__</script>
<script id="structures-cache" type="application/json">__STRUCTURES_CACHE__</script>
<script id="scop3p-cache"     type="application/json">__SCOP3P_CACHE__</script>
<script id="scop3p-compact"   type="application/json">__SCOP3P_COMPACT__</script>
<script id="biophys-compact"  type="application/json">__BIOPHYS_CACHE__</script>
</head>

<body>
<div class="hint">Top: 1D tracks. Bottom-left: mapped structure with per-residue sticks; Bottom-right: PDBe-KB superposition.</div>
<div class="warn" id="warn"></div>

<div class="grid">
  <!-- TOP: 1D full-width -->
  <div class="top-1d">
    <nightingale-manager id="mgr">
      <table>
        <tbody>
          <tr><td></td><td><nightingale-navigation id="navigation" min-width="800" height="40" length="770" display-start="1" display-end="770" margin-color="white" highlight-event="onclick"></nightingale-navigation></td></tr>
          <tr><td></td><td><nightingale-sequence id="sequence" min-width="800" height="40" length="770" display-start="1" display-end="770" margin-color="white" highlight-event="onclick"></nightingale-sequence></td></tr>
          <tr><td></td><td><nightingale-colored-sequence id="colored-sequence" min-width="800" height="15" length="770" display-start="1" display-end="770" scale="hydrophobicity-scale" margin-color="white" highlight-event="onclick"></nightingale-colored-sequence></td></tr>
          <tr><td>Domain</td><td><nightingale-track id="domain" min-width="800" height="18" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick" display-labels="true"></nightingale-track><div id="domain-legend" class="legend"></div></td></tr>
          <tr><td>Region</td><td><nightingale-track id="region" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Site</td><td><nightingale-track id="site"   min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Chain</td><td><nightingale-track id="chain"  layout="non-overlapping" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Binding site</td><td><nightingale-track id="binding" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Disulfide bond</td><td><nightingale-track id="disulfide-bond" layout="non-overlapping" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Beta strand</td><td><nightingale-track id="beta-strand" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>
          <tr><td>Scop3P phosphorylation</td><td><nightingale-track id="scop3p" min-width="800" height="15" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-track></td></tr>

          <!-- Biophysical linegraph tracks (distinct colors) -->
          <tr><td>Backbone (DynaMine)</td><td><nightingale-linegraph-track id="bb-track" min-width="800" height="46" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-linegraph-track></td></tr>
          <tr><td>Disorder (DisoMine)</td><td><nightingale-linegraph-track id="do-track" min-width="800" height="46" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-linegraph-track></td></tr>
          <tr><td>Early folding (EFoldMine)</td><td><nightingale-linegraph-track id="ef-track" min-width="800" height="46" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-linegraph-track></td></tr>

          <tr><td>Variants</td><td><nightingale-linegraph-track id="variants" min-width="800" height="60" length="770" display-start="1" display-end="770" margin-color="aliceblue" highlight-event="onclick"></nightingale-linegraph-track></td></tr>
        </tbody>
      </table>
    </nightingale-manager>
  </div>

  <!-- CENTER: toggle -->
  <div class="center-toggle">
    <label><input type="checkbox" id="toggleSuperpose" disabled> Show superposition (PDBe-KB)</label>
  </div>

  <!-- BOTTOM: two 3D panels -->
  <div class="bottom-3d">

    <!-- Bottom-left: Mapped structure -->
    <div class="panel">
      <div class="titlebar">
        <h3>Mapped structure</h3>
        <div><button id="btnFitMain" class="btn" type="button">Fit</button></div>
      </div>
      <div class="controls">
        <label>Structure <select id="structureSelect" style="margin-left:6px;"></select></label>
        <label style="margin-left:12px;"><input type="checkbox" id="toggleScop3P" checked> Show Scop3P sites</label>
        <label style="margin-left:12px;"><input type="checkbox" id="toggleBioColor"> Color by</label>
        <select id="bioColorProp" disabled>
          <option value="backbone">Backbone</option>
          <option value="disorder">Disorder</option>
          <option value="efold">Early folding</option>
        </select>
      </div>
      <div id="pdbeMolstarMain" class="viewer"></div>
    </div>

    <!-- Bottom-right: Superposition -->
    <div class="panel">
      <div class="titlebar">
        <h3>Superposition</h3>
        <div>
          <button id="btnPopOut" class="btn" type="button" title="Open in a new window">Pop-out</button>
          <button id="btnHideSuper" class="btn" type="button" title="Hide the superposition panel">Hide</button>
        </div>
      </div>

      <div class="controls">
        <label>Segment <select id="segmentSelect" disabled></select></label>
        <label>Clusters <select id="clusterSelect" multiple size="3" disabled style="min-width:160px;"></select></label>
        <label><input type="checkbox" id="toggleCompleteCluster" disabled> Complete cluster</label>
        <label><input type="checkbox" id="toggleLigandView" disabled> Ligand view</label>
        <label title="Ligand color"><input type="color" id="ligandColor" value="#ffff32" disabled></label>
        <span class="flex-spacer"></span>
        <label title="Highlight Scop3P phospho sites (sticks) within the segment"><input type="checkbox" id="toggleScop3P_super" checked disabled> Scop3P sites</label>
        <span id="superHint" class="hint" style="padding:0 0 0 8px;"></span>
      </div>

      <div id="pdbeMolstarSuper" class="viewer"></div>
    </div>

  </div>
</div>

<script type="module">
  import "@nightingale-elements/nightingale-sequence@latest";
  import "@nightingale-elements/nightingale-track@latest";
  import "@nightingale-elements/nightingale-manager@latest";
  import "@nightingale-elements/nightingale-navigation@latest";
  import "@nightingale-elements/nightingale-colored-sequence@latest";
  import "@nightingale-elements/nightingale-linegraph-track@latest";

  const accession = "__ACC__";
  const featuresCache   = JSON.parse(document.getElementById("features-cache").textContent || "{}");
  const variationCache  = JSON.parse(document.getElementById("variation-cache").textContent || "{}");
  const structuresCache = JSON.parse(document.getElementById("structures-cache").textContent || "{}");
  const scop3pCache     = JSON.parse(document.getElementById("scop3p-cache").textContent || "{}");
  const biophysCache    = JSON.parse(document.getElementById("biophys-compact").textContent || "{}");

  const featuresData  = featuresCache[accession];
  const variationData = variationCache[accession] || { features: [] };
  const structures    = structuresCache[accession] || [{ id: "AF-" + accession + "-F1", source:"AFDB", chain:null, label:"AlphaFold (AF-" + accession + "-F1)" }];

  await Promise.all([
    customElements.whenDefined("nightingale-sequence"),
    customElements.whenDefined("nightingale-navigation"),
    customElements.whenDefined("nightingale-manager"),
    customElements.whenDefined("nightingale-track"),
    customElements.whenDefined("nightingale-colored-sequence"),
    customElements.whenDefined("nightingale-linegraph-track")
  ]);

  const warnEl = document.getElementById("warn");
  const mgr = document.getElementById("mgr");
  const navEl = document.getElementById("navigation");
  const seqEl = document.getElementById("sequence");
  const coloredEl = document.getElementById("colored-sequence");
  const trDomain = document.getElementById("domain");
  const domainLegend = document.getElementById("domain-legend");
  const trRegion = document.getElementById("region");
  const trSite = document.getElementById("site");
  const trBinding = document.getElementById("binding");
  const trChain = document.getElementById("chain");
  const trDisulf = document.getElementById("disulfide-bond");
  const trStrand = document.getElementById("beta-strand");
  const trScop3P = document.getElementById("scop3p");
  const variantsEl = document.getElementById("variants");

  const bbTrack = document.getElementById("bb-track");
  const doTrack = document.getElementById("do-track");
  const efTrack = document.getElementById("ef-track");

  const structSelect = document.getElementById("structureSelect");
  const toggleScop = document.getElementById("toggleScop3P");

  const toggleBioColor = document.getElementById("toggleBioColor");
  const bioColorProp   = document.getElementById("bioColorProp");

  const toggleSuper = document.getElementById("toggleSuperpose");
  const segSel = document.getElementById("segmentSelect");
  const cluSel = document.getElementById("clusterSelect");
  const chkComplete = document.getElementById("toggleCompleteCluster");
  const chkLigand = document.getElementById("toggleLigandView");
  const ligandColor = document.getElementById("ligandColor");
  const toggleScopSuper = document.getElementById("toggleScop3P_super");
  const superHint = document.getElementById("superHint");

  const mainDiv  = document.getElementById("pdbeMolstarMain");
  const superDiv = document.getElementById("pdbeMolstarSuper");
  const btnPopOut = document.getElementById("btnPopOut");
  const btnHideSuper = document.getElementById("btnHideSuper");
  const btnFitMain = document.getElementById("btnFitMain");

  let segmentsMeta = [];

  // 1D lengths
  const seq = featuresData.sequence;
  const len = seq.length;
  const setLen = (el) => { if (!el) return; el.length = len; el.setAttribute("length", String(len)); el.displayStart = 1; el.displayEnd = len; el.setAttribute("display-start","1"); el.setAttribute("display-end", String(len)); };
  [navEl, seqEl, coloredEl, trDomain, trRegion, trSite, trBinding, trChain, trDisulf, trStrand, trScop3P, bbTrack, doTrack, efTrack, variantsEl].forEach(setLen);

  seqEl.data = seq;
  coloredEl.data = seq;

  const feats = (featuresData.features || []).map(ft => ({ ...ft, start: ft.start ?? ft.begin }));
  trDomain.data = feats.filter(f => f.type === "DOMAIN").map(f => ({ ...f, label: f.description || f.type || "domain" }));
  trRegion.data = feats.filter(f => f.type === "REGION");
  trSite.data   = feats.filter(f => f.type === "SITE");
  trBinding.data= feats.filter(f => f.type === "BINDING");
  trChain.data  = feats.filter(f => f.type === "CHAIN");
  trDisulf.data = feats.filter(f => f.type === "DISULFID");
  trStrand.data = feats.filter(f => f.type === "STRAND");

  try {
    const items = trDomain.data.slice(0, 12).map(d => `${d.label || d.description || "domain"} (${d.start}-${d.end})`);
    domainLegend.textContent = items.length ? "Domains: " + items.join(" · ") + (trDomain.data.length > 12 ? " …" : "") : "";
  } catch(e) {}

  const counts = new Map();
  for (const v of (variationData.features || [])) {
    const pos = Number(v.begin);
    if (!Number.isFinite(pos)) continue;
    counts.set(pos, (counts.get(pos) || 0) + 1);
  }
  const values = Array.from(counts, ([position, value]) => ({ position, value })).sort((a,b)=>a.position-b.position);
  const max = values.length ? Math.max(...values.map(d=>d.value)) : 0;
  variantsEl.data = [{ color: "grey", values, range: [0, Math.max(1, max)] }];

  trScop3P.data = (scop3pCache[accession]?.modifications || [])
    .filter(m => String(m.name).toLowerCase()==="phosphorylation")
    .filter(m => Number.isFinite(Number(m.position)))
    .map(m => ({ start: +m.position, end: +m.position, type: "SCOP3P_PHOSPHO", description: `Phospho ${(m.residue||'?').slice(0,1)}@${m.position} (${m.source||'Scop3P'})` }));

  // === Biophys tracks (line colors) ===
  const biophys = biophysCache[accession] || {};
  const hasBiophys = Array.isArray(biophys.backbone) && biophys.backbone.length === len;

  function asLineValues(arr) { return arr.map((v, i) => ({ position: i + 1, value: Number.isFinite(v) ? v : null })); }

  if (hasBiophys) {
    bbTrack.data = [{ color: "#d62728", values: asLineValues(biophys.backbone), range: biophys.ranges?.backbone || [0, 1.2] }];
    doTrack.data = [{ color: "#1f77b4", values: asLineValues(biophys.disorder), range: biophys.ranges?.disorder || [0, 1] }];
    efTrack.data = [{ color: "#2ca02c", values: asLineValues(biophys.efold),   range: biophys.ranges?.efold    || [0, 0.5] }];
    document.getElementById("bioColorProp").disabled = false;
  } else {
    bbTrack.style.display = "none"; doTrack.style.display = "none"; efTrack.style.display = "none";
    document.getElementById("toggleBioColor").disabled = true; document.getElementById("bioColorProp").disabled = true;
  }

  // Populate structures
  function populateStructureSelect(arr) {
    const el = structSelect;
    el.innerHTML = "";
    const afFirst = arr.slice().sort((a,b) => (a.source === "AFDB" ? -1 : (b.source === "AFDB" ? 1 : 0)));
    for (const s of afFirst) {
      const opt = document.createElement("option");
      opt.value = (s.source === "PDB" && s.chain) ? `${s.id}|${s.chain}` : s.id;
      opt.textContent = s.label || (s.source === "PDB" ? `${s.id}${s.chain? " — chain "+s.chain : ""}` : s.id);
      opt.dataset.source = s.source || "";
      el.appendChild(opt);
    }
    const afIdx = Array.from(el.options).findIndex(o => o.value.startsWith(`AF-${accession}-F1`));
    el.selectedIndex = afIdx >= 0 ? afIdx : 0;
  }
  populateStructureSelect(structures);

  // Mol* plugin
  const PDBeMolstarPlugin = window.PDBeMolstarPlugin;
  let viewerMain = null;
  let viewerSuper = null;

  function waitForNonZeroSize(el) {
    return new Promise((resolve) => {
      let ok = 0;
      const check = () => {
        const r = el.getBoundingClientRect();
        if (r.width > 0 && r.height > 0) { ok++; if (ok >= 2) return resolve(); }
        else ok = 0;
        requestAnimationFrame(check);
      };
      const ro = new ResizeObserver(() => {
        const r = el.getBoundingClientRect();
        if (r.width > 0 && r.height > 0) { ok++; if (ok >= 2) { ro.disconnect(); resolve(); } }
      });
      ro.observe(el);
      check();
    });
  }
  async function renderInto(container, options, which) {
    container.innerHTML = "";
    await waitForNonZeroSize(container);
    try { const v = which === "main" ? viewerMain : viewerSuper; v?.plugin?.dispose && v.plugin.dispose(); } catch {}
    const inst = new PDBeMolstarPlugin();
    inst.render(container, options);
    if (which === "main") viewerMain = inst; else viewerSuper = inst;
    return inst;
  }

  // Sticks/highlights
  function buildScop3PSelection() {
    const mods = (scop3pCache[accession]?.modifications || [])
      .filter(m => String(m.name).toLowerCase() === "phosphorylation")
      .filter(m => Number.isFinite(Number(m.position)));
    return mods.map(m => ({
      uniprot_accession: accession,
      start_uniprot_residue_number: +m.position,
      end_uniprot_residue_number: +m.position,
      color: { r: 255, g: 0, b: 0 },
      representation: "ball-and-stick",
      sideChain: true
    }));
  }
  function buildClickSelection(rangeStr) {
    if (!rangeStr) return [];
    return rangeStr.split(",").filter(Boolean).map(seg => {
      const [a,b] = seg.split(":").map(Number);
      return {
        uniprot_accession: accession,
        start_uniprot_residue_number: Math.min(a,b),
        end_uniprot_residue_number: Math.max(a,b),
        color: { r: 255, g: 221, b: 0 },
        representation: "ball-and-stick",
        sideChain: true
      };
    });
  }
  function applyOverlays() {
    const v = viewerMain;
    if (!v?.visual?.select) return;
    const navH = navEl.getAttribute("highlight") || "";
    const clickData = buildClickSelection(navH);
    const scopData = toggleScop.checked ? buildScop3PSelection() : [];
    const combined = [...scopData, ...clickData];
    const apply = () => v.visual.select({ data: combined, nonSelectedColor: { r: 176, g: 176, b: 176 } });
    try { apply(); } catch { setTimeout(() => { try { apply(); } catch {} }, 300); }
  }

  // Load main (AF with alphafoldView, PDB by moleculeId for SIFTS mapping)
  async function loadMain() {
    const opt = structSelect.options[structSelect.selectedIndex];
    const value = opt ? opt.value : "";
    const source = opt ? (opt.dataset.source || "") : "";

    if (source === "AFDB" || value.startsWith(`AF-${accession}-F1`)) {
      const url = `https://alphafold.ebi.ac.uk/files/AF-${accession}-F1-model_v6.cif`;
      await renderInto(mainDiv, {
        customData: { url, format: "cif", binary: false },
        hideControls: true,
        bgColor: { r: 255, g: 255, b: 255 },
        alphafoldView: true
      }, "main");
    } else {
      const pdbId = (value.split("|")[0] || "").toLowerCase();
      if (!pdbId) return;
      await renderInto(mainDiv, {
        moleculeId: pdbId,
        hideControls: true,
        bgColor: { r: 255, g: 255, b: 255 }
      }, "main");
    }

    const reassert = () => {
      if (toggleBioColor.checked && hasBiophys) setTimeout(() => applyGradientColor(bioColorProp.value), 50);
      applyOverlays();
    };
    if (viewerMain?.events?.loadComplete?.subscribe) {
      const sub = viewerMain.events.loadComplete.subscribe(() => { reassert(); setTimeout(reassert, 200); setTimeout(reassert, 500); try{ sub.unsubscribe(); }catch{} });
    } else { setTimeout(reassert, 400); setTimeout(reassert, 800); }
  }

  // ===== Gradient (red→blue) per residue, Scop3P-style sticks =====
  function clamp01(x){ return x < 0 ? 0 : (x > 1 ? 1 : x); }
  function lerp(a,b,t){ return a + (b - a) * t; }
  function redToBlue(val, vmin, vmax){
    const t = (vmax > vmin) ? clamp01((val - vmin) / (vmax - vmin)) : 0;
    return { r: Math.round(lerp(255, 0, t)), g: 0, b: Math.round(lerp(0, 255, t)) };
  }
  function finiteMinMax(arr){
    let mn = +Infinity, mx = -Infinity;
    for (const x of arr) if (Number.isFinite(x)) { if (x < mn) mn = x; if (x > mx) mx = x; }
    if (!Number.isFinite(mn) || !Number.isFinite(mx)) { mn = 0; mx = 1; }
    if (mx === mn) mx = mn + 1e-6;
    return [mn, mx];
  }

  function applyGradientColor(prop){
    if (!viewerMain?.visual?.select || !hasBiophys) return;

    let arr = null;
    if (prop === "backbone") arr = biophys.backbone;
    else if (prop === "disorder") arr = biophys.disorder;
    else if (prop === "efold") arr = biophys.efold;
    else return;

    const [vmin, vmax] = finiteMinMax(arr);

    const data = [];
    for (let i = 0; i < len; i++) {
      const v = Number(arr[i]);
      const c = Number.isFinite(v) ? redToBlue(v, vmin, vmax) : { r: 176, g: 176, b: 176 };
      data.push({
        uniprot_accession: accession,
        start_uniprot_residue_number: i + 1,
        end_uniprot_residue_number: i + 1,
        color: c,
        representation: "cartoon",   // <-- same as Scop3P, guaranteed to recolor
        sideChain: false
      });
    }

    try {
      viewerMain.visual.select({ data, nonSelectedColor: { r: 200, g: 200, b: 200 } });
    } catch(_) {}
  }

  function clearBioColor(){
    if (!viewerMain?.visual?.select) return;
    try { viewerMain.visual.select({ data: [], nonSelectedColor: { r: 176, g: 176, b: 176 } }); } catch(_) {}
    // leave Scop3P sticks as they were
    applyOverlays();
  }

  // ===== Superposition (unchanged) =====
  async function fetchSuperpositionGraph(uniprot) {
    try {
      const url = "https://www.ebi.ac.uk/pdbe/graph-api/uniprot/superposition/" + uniprot;
      const res = await fetch(url, { cache: "no-store" });
      if (!res || !res.ok) return [];
      const js = await res.json();
      const arr = js && js[uniprot] ? js[uniprot] : [];
      if (!Array.isArray(arr)) return [];
      const out = [];
      for (let i=0; i<arr.length; i++) {
        const seg = arr[i];
        const segId = i + 1;
        const clusters = [];
        const raws = Array.isArray(seg.clusters) ? seg.clusters : [];
        for (let k=0; k<raws.length; k++) {
          const list = raws[k];
          clusters.push({ id: k + 1, size: Array.isArray(list) ? list.length : 0, repr: Array.isArray(list) ? list.find(x => x?.is_representative) : null });
        }
        out.push({ id: segId, start: +seg.segment_start, end: +seg.segment_end, clusters });
      }
      return out;
    } catch { return []; }
  }
  function populateSegmentsUI(segments) {
    segSel.innerHTML = "";
    for (const s of segments) {
      const opt = document.createElement("option");
      opt.value = String(s.id);
      opt.textContent = `Segment ${s.id} (${s.start}-${s.end})`;
      segSel.appendChild(opt);
    }
    segSel.disabled = segments.length === 0;
  }
  function populateClustersUI(segments, segId) {
    const s = segments.find(x => x.id == segId);
    const cls = (s && s.clusters) ? s.clusters : [];
    cluSel.innerHTML = "";
    for (const c of cls) {
      const opt = document.createElement("option");
      const reprTxt = (c.repr && c.repr.pdb_id) ? ` repr ${c.repr.pdb_id}:${(c.repr.auth_asym_id || c.repr.struct_asym_id || "")}` : "";
      opt.value = String(c.id);
      opt.textContent = `Cluster ${c.id} (n=${c.size})${reprTxt}`;
      cluSel.appendChild(opt);
    }
    cluSel.disabled = cls.length === 0;
  }
  function buildSuperpositionOptions({ matrixAccession, segment, clusters, completeCluster, ligandView, ligandColor, superposeAll }) {
    const seg = Number(segment);
    if (!Number.isFinite(seg) || seg < 1) throw new Error("Invalid segment index");
    const opts = { moleculeId: String(matrixAccession || accession).toUpperCase(), hideControls: true, bgColor:{ r:255,g:255,b:255 }, superposition:true, superpositionParams:{ matrixAccession:String(matrixAccession||"").toUpperCase(), segment:seg, superposeAll: !!superposeAll } };
    const cls = Array.isArray(clusters) ? clusters.map(n => Number(n)).filter(n => Number.isFinite(n) && n >= 1) : [];
    if (cls.length) opts.superpositionParams.cluster = cls;
    if (completeCluster != null) opts.superpositionParams.superposeCompleteCluster = !!completeCluster;
    if (ligandView) { opts.superpositionParams.ligandView = true; if (ligandColor && /^#?[0-9a-f]{6}$/i.test(ligandColor)) opts.superpositionParams.ligandColor = ligandColor.startsWith("#") ? ligandColor : ("#"+ligandColor); }
    return opts;
  }
  function buildScop3PSelectionInRange(start, end) {
    const mods = (scop3pCache[accession]?.modifications || [])
      .filter(m => String(m.name).toLowerCase() === "phosphorylation")
      .map(m => ({ pos: +m.position, residue: (m.residue || "?")[0] }))
      .filter(m => Number.isFinite(m.pos) && m.pos >= start && m.pos <= end);
    return mods.map(m => ({ uniprot_accession: accession, start_uniprot_residue_number: m.pos, end_uniprot_residue_number: m.pos, color: { r: 255, g: 0, b: 0 }, representation: "ball-and-stick", sideChain: true }));
  }
  function applySuperOverlays(segId) {
    if (!viewerSuper?.visual?.select) return;
    const seg = segmentsMeta.find(s => s.id == segId); if (!seg) return;
    const data = (toggleScopSuper?.checked) ? buildScop3PSelectionInRange(seg.start, seg.end) : [];
    const apply = () => viewerSuper.visual.select({ data, nonSelectedColor: { r: 176, g: 176, b: 176 } });
    try { apply(); } catch { setTimeout(() => { try { apply(); } catch {} }, 300); }
  }
  async function renderSuperposition(segments) {
    [segSel, cluSel, chkComplete, chkLigand, ligandColor, toggleScopSuper].forEach(el => el.disabled = false);
    let segId = parseInt(segSel.value, 10);
    if (!Number.isFinite(segId) && segments.length) { segId = segments[0].id; segSel.value = String(segId); }
    if (!Number.isFinite(segId)) { warnEl.textContent = "No valid segment available for superposition."; toggleSuper.checked = false; superDiv.style.display = "none"; try { viewerSuper?.plugin?.dispose?.(); } catch {} viewerSuper = null; return; }
    const clusters = Array.from(cluSel.selectedOptions).map(o => parseInt(o.value, 10)).filter(Number.isFinite);
    const col = ligandColor.value || "#ffff32";
    const options = buildSuperpositionOptions({ matrixAccession: accession, segment: segId, clusters, completeCluster: chkComplete.checked, ligandView: chkLigand.checked, ligandColor: col, superposeAll: clusters.length === 0 });
    superDiv.style.display = "block";
    await renderInto(superDiv, options, "super");
    superHint.textContent = clusters.length ? `Segment ${segId}, clusters [${clusters.join(", ")}]` : `Segment ${segId} (all)`;
    applySuperOverlays(segId);
    if (viewerSuper?.events?.loadComplete?.subscribe) {
      const sub2 = viewerSuper.events.loadComplete.subscribe(() => { applySuperOverlays(segId); try { sub2.unsubscribe(); } catch {} });
    }
  }

  // Events
  structSelect.addEventListener("change", loadMain);
  toggleScop.addEventListener("change", () => applyOverlays());
  const obs = new MutationObserver(() => applyOverlays()); obs.observe(navEl, { attributes: true, attributeFilter: ["highlight"] });
  mgr.addEventListener("click", () => applyOverlays());
  btnFitMain.addEventListener("click", () => { try { viewerMain?.visual?.reset({ camera: true }); } catch {} });

  toggleBioColor.addEventListener("change", () => {
    if (!hasBiophys) return;
    bioColorProp.disabled = !toggleBioColor.checked;
    if (toggleBioColor.checked) applyGradientColor(bioColorProp.value);
    else clearBioColor();
  });
  bioColorProp.addEventListener("change", () => { if (toggleBioColor.checked) applyGradientColor(bioColorProp.value); });

  btnHideSuper.addEventListener("click", () => {
    toggleSuper.checked = false; superDiv.style.display = "none"; superHint.textContent = "";
    try { viewerSuper?.plugin?.dispose?.(); } catch {} viewerSuper = null;
  });
  btnPopOut.addEventListener("click", () => {
    if (!toggleSuper.checked) return;
    const segId = segSel.value || "1";
    const clusters = Array.from(cluSel.selectedOptions).map(o => o.value);
    const params = { matrixAccession: accession, segment: segId, cluster: clusters.join(","), complete: chkComplete.checked ? "1" : "0", ligand: chkLigand.checked ? "1" : "0", color: ligandColor.value.replace("#","") };
    const q = new URLSearchParams(params).toString();
    const w = window.open("", "_blank", "width=960,height=720"); if (!w) return;
    const popupHtml =
      '<!doctype html><meta charset="utf-8">' +
      '<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/pdbe-molstar@3.8.0/build/pdbe-molstar.css">' +
      '<script src="https://cdn.jsdelivr.net/npm/pdbe-molstar@3.8.0/build/pdbe-molstar-plugin.js"><\\/script>' +
      '<body style="margin:0"><div id="v" style="position:fixed;inset:0"></div>' +
      '<script>(function(){var p=new URLSearchParams("'+q+'");var acc=p.get("matrixAccession");var seg=Number(p.get("segment"));var cls=(p.get("cluster")||"").split(",").map(Number).filter(function(n){return Number.isFinite(n)&&n>0});var complete=p.get("complete")==="1";var ligand=p.get("ligand")==="1";var color=p.get("color");var opts={moleculeId: acc, hideControls:true,bgColor:{r:255,g:255,b:255},superposition:true,superpositionParams:{matrixAccession:acc,segment:seg,superposeAll:!cls.length}};if(cls.length)opts.superpositionParams.cluster=cls;if(complete)opts.superpositionParams.superposeCompleteCluster=true;if(ligand){opts.superpositionParams.ligandView=true;if(color)opts.superpositionParams.ligandColor="#"+color;}var el=document.getElementById("v");function wait(){var r=el.getBoundingClientRect();if(r.width>0&&r.height>0){new window.PDBeMolstarPlugin().render(el,opts);}else{requestAnimationFrame(wait);}}wait();})();<\\/script></body>';
    w.document.open(); w.document.write(popupHtml); w.document.close();
  });
  toggleSuper.addEventListener("change", () => { if (toggleSuper.checked) { renderSuperposition(segmentsMeta); superDiv.style.display = "block"; } else { btnHideSuper.click(); } });
  toggleScopSuper.addEventListener("change", () => { if (toggleSuper.checked) applySuperOverlays(segSel.value); });
  segSel.addEventListener("change", () => { populateClustersUI(segmentsMeta, segSel.value); if (toggleSuper.checked) renderSuperposition(segmentsMeta); });
  cluSel.addEventListener("change", () => { if (toggleSuper.checked) renderSuperposition(segmentsMeta); });
  chkComplete.addEventListener("change", () => { if (toggleSuper.checked) renderSuperposition(segmentsMeta); });
  chkLigand.addEventListener("change", () => { if (toggleSuper.checked) renderSuperposition(segmentsMeta); });
  ligandColor.addEventListener("input", () => { if (toggleSuper.checked) renderSuperposition(segmentsMeta); });

  // Init
  await loadMain();
  const segments = await fetchSuperpositionGraph(accession);
  segmentsMeta = segments;

  if (!segments.length) {
    toggleSuper.disabled = true;
    [segSel, cluSel, chkComplete, chkLigand, ligandColor, toggleScopSuper].forEach(el => el.disabled = true);
    warnEl.textContent = "Superposition not available for " + accession + " (no PDBe-KB segments).";
  } else {
    toggleSuper.disabled = false;
    warnEl.textContent = "";
    populateSegmentsUI(segments);
    segSel.value = String(segments[0].id);
    populateClustersUI(segments, segSel.value);
  }
</script>
</body>
</html>
"""
def build_html(acc: str, out_dir: str = ".") -> Path:
    b = build_bundle(acc)
    features_cache   = json.dumps({acc: b["features"]}, ensure_ascii=False)
    variation_cache  = json.dumps({acc: b["variation"]}, ensure_ascii=False)
    structures_cache = json.dumps({acc: b["structures"]}, ensure_ascii=False)
    scop3p_cache     = json.dumps({acc: b["scop3p"]}, ensure_ascii=False)
    scop3p_compact   = json.dumps({acc: b["scop3p_compact"]}, ensure_ascii=False)
    biophys_cache    = json.dumps({acc: b["biophys_compact"]}, ensure_ascii=False)

    html_out = (
        HTML_TEMPLATE
        .replace("__ACC__", acc)
        .replace("__FEATURES_CACHE__", features_cache)
        .replace("__VARIATION_CACHE__", variation_cache)
        .replace("__STRUCTURES_CACHE__", structures_cache)
        .replace("__SCOP3P_CACHE__", scop3p_cache)
        .replace("__SCOP3P_COMPACT__", scop3p_compact)
        .replace("__BIOPHYS_CACHE__", biophys_cache)
    )
    out_path = Path(out_dir) / f"nightingale_pdbe_backbone_biophys_{acc}.html"
    out_path.write_text(html_out, encoding="utf-8")
    return out_path


In [62]:
p = build_html("P07949")
p

PosixPath('nightingale_pdbe_backbone_biophys_P07949.html')

In [ ]:
# RINs using RIN maker